<h1>Limpieza y análisis exploratorio de la base de datos Registro Mercantil de la Cámara de Comercio de Cali </h1>

In [1]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


ModuleNotFoundError: No module named 'pandas_profiling'

In [2]:
pip install openpyxl==3.0.9

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Se lee la base de datos
# IMPORTANTE: se debe cambiar el nombre de la base de datos al igual que la dirección URL. Preferiblemente que los datos estén en la misma carpeta que el notebook (.ipynb)
# Esta línea es para leerlos en formato EXCEL
df = pd.read_excel('MATREM_CCC_ENE-AGO2022_V1(SEP_20_EEE3).xlsx',  dtype={'CODIGO_CIIU':object, 'MATRICULA': object, 'NIT':object, 'Año de FECHA_RENOVACION':object, 'TELEFONO': object})
#df = pd.read_excel('RM_2018_V1_2.xlsx', dtype={'CODIGO_CIIU':object, 'MATRICULA': object, 'NIT':object, 'Año de FECHA_RENOVACION':object, 'PATRIMONIO': np.int, 'UTILIDAD_PERDIDA': np.int, 'VENTA': np.int, 'UTILIDAD_BRUTA': np.int, 'ESTABLECIMIENTOS': np.int})
# Esta línea es para leerlos en formato CSV especificando la clase de los objetos que necesitamos en la base
# df = pd.read_csv('C:/Users/ortiz/Downloads/BD_Marketing.csv', encoding='latin-1', dtype={'CODIGO_CIIU':object, 'MATRICULA': object, 'CIIU_COR': object, 'DIVISION': object, 'GRUPO': object})
pd.set_option('display.max_columns', None)
print(df.info())
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108464 entries, 0 to 108463
Data columns (total 39 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   MATRICULA                108464 non-null  object        
 1   NIT                      108464 non-null  object        
 2   ESTADO_MAT_REN           108464 non-null  object        
 3   MAT_REN                  108464 non-null  object        
 4   DESC_CATEGORIA           108464 non-null  object        
 5   ESTADO_ACTUAL            108464 non-null  object        
 6   SECTOR                   108464 non-null  object        
 7   RAZON_SOCIAL             108460 non-null  object        
 8   DESC_CIIU                108464 non-null  object        
 9   NOM_ENTE_JURIDICO        108464 non-null  object        
 10  DESC_TAMANO              108464 non-null  object        
 11  TAMANO957                108464 non-null  object        
 12  CIUDAD          

,MATRICULA,NIT,ESTADO_MAT_REN,MAT_REN,DESC_CATEGORIA,ESTADO_ACTUAL,SECTOR,RAZON_SOCIAL,DESC_CIIU,NOM_ENTE_JURIDICO,DESC_TAMANO,TAMANO957,CIUDAD,Año de FECHA_RENOVACION,FECHA_RENOVACION,FECHA_MATRICULA,FECHA_MAT_REN,EMAIL,TELEFONO,COMUNA,DIRECCION_COMERCIAL,DESC_CATEGORIA.1,DESC_TIPO_REGISTRO,PATRIMONIO,PERSONAL_OCUPADO,ESTABLECIMIENTOS,TOT_ACTIVOS,TOT_PASIVO,UTILILIDAD_PERDIDA,UTILILIDAD_BRUTA,VENTAS,CONSECUTIVO_PROPIETARIO,NUM_CONSECUTIVO,CODIGO_CIIU,BARRIO,REP_LEGAL,AFILIADO,PORCENT_EXTRANJERO,Sin valor de medida
0,60,890306499,A,Renovacion,PERSONA JURIDICA,ACTIVO,C,UNION PLASTICA S.A.S. ...,FABRICACIÓN DE ARTÍCULOS DE PLÁSTICO N.C.P. ...,Sociedad por Acciones Simplificada ...,GRANDE,MEDIANA,Yumbo,2022,2022-03-11,1972-01-12,2022-03-11 09:18:15,uniplas@unionplastica.com,6644425,Comuna Yumbo,KR 39 # 12 A - 151 ACOPI ...,PERSONA JURIDICA,MERCANTIL,2.401912e+10,193.0,1,38303902705,1.428478e+10,6364618950,4.257448e+09,4.221629e+10,0.0,19,2229,Barrio Yumbo,JOSÉ GUILLERMO PARDO BORRERO,S,0.0,NaN
1,64,860026759,A,Renovacion,PERSONA JURIDICA,ACTIVO,C,CARTONES AMERICA S.A. CAME ...,FABRICACIÓN DE PAPEL Y CARTÓN ONDULADO (CORRUG...,Sociedad Anónima ...,GRANDE,GRANDE,Cali,2022,2022-03-24,1972-01-12,2022-03-24 11:15:30,evelyn.bustos@cartonesamerica.com,6023876990,Comuna 02,CL 70 NORTE # 2 A - 130 ...,PERSONA JURIDICA,MERCANTIL,3.048232e+11,435.0,2,571508138000,2.666849e+11,79168212000,9.903336e+10,3.988043e+11,0.0,21,1702,Brisas De Los Alamos,JUAN ADALBERTO ZOLTAN ANDUJAR,S,30.0,NaN


In [5]:
# Se cambian los nombres de las variables para facilitar su interpretación
df.rename(columns={'Año de FECHA_RENOVACION':'ULTIMO_ANO_RENOVADO',
                          'FECHA_MAT_REN':'FECHA_MATRICULA_RENOVACION_HORA',
                          'TOT_ACTIVOS':'ACTIVO_TOTAL',
                          'UTILILIDAD_PERDIDA': 'UTILILIDAD_OPERACIONAL',
                          'VENTAS' : 'INGRESOS_ACTIVIDAD_ORDINARIA',
                          'TOT_PASIVO': 'PASIVO_TOTAL',
                          'UTILILIDAD_BRUTA': 'RESULTADO_PERIODO',
                          'MAT_REN' : 'MATRICULA_RENOVACION'}, 
                 inplace=True)

In [6]:
# Se separa la fecha para facilitar el análisis
df[['FECHA_MATRICULA_RENOVACION','HORA_FECHA_MAT_REN']] =  df.FECHA_MATRICULA_RENOVACION_HORA.str.split(' ', n=1, expand=True)

In [ ]:
# Se separa día, mes y año para facilitar el análisis
df[['DIA','MES', 'ANO']] =  df.FECHA_MATRICULA_RENOVACION.str.split('/', n=3, expand=True)
#Se convierte la variable FECHA_MATRICULA_RENOVACION_HORA en datetime para facilitar manupulación de los datos
df.FECHA_MATRICULA_RENOVACION_HORA = pd.to_datetime(df.FECHA_MATRICULA_RENOVACION_HORA)

In [ ]:
#Se convierte la variable FECHA_MATRICULA en datetime para facilitar manupulación de los datos
df.FECHA_MATRICULA = pd.to_datetime(df.FECHA_MATRICULA)

In [ ]:
#Se organiza el DataFrame según la fecha del movimiento registrado. Esto facilitará las particiones que se harán de la base de datos
df=df.sort_values('FECHA_MATRICULA_RENOVACION_HORA')
print(df.info())
df.head(2)

In [ ]:
df.groupby('ESTADO_MAT_REN').nunique()

In [ ]:
# Siguiendo las indicaciones de Sistemas y Registro se deben eliminar las filas de las empresas que registren en el ESTADO_MAT_REN la letra R ya que estos movimientos fueron anulados por alguna acción legal. En la mayoría de los casos se anulan las renovaciones por motivo de cancelación hechas antes del 31 de marzo de cada año ya que la ley permite que las empresas cancelen su registro sin pagar la renovación solo si se realiza el proceso antes del 31 de marzo de cda año
#df = df[df.ESTADO_MAT_REN != 'R']

In [ ]:
# Para obtener los duplicados EXCEPTO la primera observación revisando todas las columnas 
duplicados = df[df.duplicated()]
print('El número de registros duplicados con todas las columnas iguales son:')
print (duplicados.MATRICULA.count())

In [ ]:
# Para obtener los duplicados EXCEPTO la última observación revisando la columna Matriculados. Es decir, se tomarán todas las matriculas duplicadas excepto la que registró la ultima modificación 
duplicadosMAT = df[df.duplicated(['MATRICULA'], keep='last')]
print("El número de Matriculas duplicadas es:", duplicadosMAT.MATRICULA.count(), sep='\n')

In [ ]:
# Para obtener los duplicados EXCEPTO la última observación revisando en simultaneo las columnas Matriculados y MATRICULA_RENOVACION. Es decir, se tomarán todas las matriculas duplicadas tanto con su número de matricula como lo que hayan hecho en ese año, es decir, si presentó dos renovaciones o dos matriculas se tomarán en la subase creada. Es necesario acalarar que lo anterior se hace exceptuando la que registró la ultima modificación. Esto puede ocurrir debido a que los tramites de renovación de alguna empresa estuveiron mal y se tardaron más del plazo para entregar sus papeles así que se registra una nueva fecha de renovación sin eliminar la primera.
duplicadosMAT_REN = df[df.duplicated(['MATRICULA', 'MATRICULA_RENOVACION'], keep='last')]
print("El número de filas duplicadas analizando dos columnas es:", duplicadosMAT_REN.MATRICULA.count(), sep='\n')

In [ ]:
# Antes de partir la base de datos. Se hace un conteo para saber cuantas Matriculas y Renovaciones se tienen en la base sin eliminar duplicados
df.MATRICULA_RENOVACION.value_counts()

In [ ]:
# Ahora es necesario partir la base de datos en dos para hacer la correspondiente limpieza. Se requieren tratar las empresas Matriculadas y Renovadas diferenciadas por su fecha de registro
df_matricula = df.loc[df['MATRICULA_RENOVACION'].isin(['Matricula'])]
df_renovacion = df.loc[df['MATRICULA_RENOVACION'].isin(['Renovacion'])]
print('El número de registros en la BD para Matriculas es:', df_matricula.MATRICULA_RENOVACION.value_counts(), sep='\n')
print('El número de registros en la BD para Renovaciones:', df_renovacion.MATRICULA_RENOVACION.value_counts(), sep='\n')

<p>---------------</p>

In [ ]:
# Aqui DEBE IR EL CODIGO DESPUES DE LIMPIAR EL TEMA DE RENOVACIONES MAS VIEJAS QUE LAS MATRICULAS. SE DEBEN ELIMINAR LAS RENOVACIONES CON FECHAS ANTERIORES A LAS DE LAS MATRICULAS porque esto es un indicador que la empresa hizo un translado de domicilio y se trajo todo su historial a la CCC
# Se convierte la variable FECHA_MATRICULA_RENOVACION_HORA en datetime para facilitar manupulación de los datos
df_renovacion.FECHA_MATRICULA_RENOVACION = pd.to_datetime(df_renovacion.FECHA_MATRICULA_RENOVACION)
df_renovacion = df_renovacion.loc[df_renovacion['FECHA_MATRICULA']<=df_renovacion['FECHA_MATRICULA_RENOVACION']]

In [ ]:
# Se organizan las bases según la fecha en la cual se hizo el movimiento corresspondiente a Matricual o Renovación para posterior limpieza. Vale aclarar que esta el ordenamiento es descendente, es decir, se comienza desde YYYY/01/01 00:00 hasta YYYY/12/31 23:59 (de enero a diciembre)
df_matricula=df_matricula.sort_values('FECHA_MATRICULA_RENOVACION_HORA')
df_renovacion=df_renovacion.sort_values('FECHA_MATRICULA_RENOVACION_HORA')
df_renovacion.head(2)

In [ ]:
# Se procede a eliminar los duplicados en las dos bases y la idea es dejar el último movimiento registrado. Esto se debe gracias a que se identificó que algunas empresas registran algun movimiento y por algun motivo este es devuelto, en la mayoría de los casos la corrección sugerida es hecha en el plazo para mantener la fecha inicial del movimiento, sin embargo, cuando la empresa deja pasar este plazo máximo se debe generar un nuevo movimiento lo cual genera duplicados en la información
df2_matricula=df_matricula.sort_values('FECHA_MATRICULA_RENOVACION_HORA').drop_duplicates('MATRICULA', keep='last')
df2_renovacion=df_renovacion.sort_values('FECHA_MATRICULA_RENOVACION_HORA').drop_duplicates('MATRICULA', keep='last')

In [ ]:
# Una vez eliminados lo duplicados se cuenta el número de resgitros 
print('El número de registros en la BD para Matriculas después de eliminar duplicados es:', df2_matricula.MATRICULA_RENOVACION.value_counts(), sep='\n')
print('El número de registros en la BD para Renovaciones después de eliminar duplicados es:', df2_renovacion.MATRICULA_RENOVACION.value_counts(), sep='\n')

In [ ]:
# Se procede a unir las bases de datos para la posterior limpieza en conjunto
df2 = pd.concat([df2_matricula, df2_renovacion])
df2 = df2.sort_values('MATRICULA_RENOVACION', ascending=True)
print('El número de registros en la BD unida es:', df2.MATRICULA_RENOVACION.value_counts(), sep='\n')

In [ ]:
# Para eliminar duplicados se debe tener en cuenta que en Matricula nos interesa dejar el primer registro que tengamos en la base pues por cada matricula se genera una renovación
df3 = df2.sort_values('MATRICULA_RENOVACION', ascending=True).drop_duplicates('MATRICULA', keep='first')
print('El número de registros en la BD unida sin duplicados es:', df3.MATRICULA_RENOVACION.value_counts(), sep='\n')

In [ ]:
duplicadosMAT3 = df3[df3.duplicated(['MATRICULA'], keep='last')]
print("El número de Matriculas duplicadas es:", duplicadosMAT3.MATRICULA.count(), sep='\n')

<p> Desde aquí se inicia la limpieza de otras variables para facilitar los posteriores análisis</p>

In [ ]:
df3.groupby('CIUDAD')['MATRICULA'].nunique()

In [ ]:
# En esta línea de código se limpian los diferentes valores de Comuna que se tienen
df3.COMUNA.replace(to_replace =["Comuna 01", "Comuna 02", "Comuna 03", "Comuna 04", "Comuna 05", "Comuna 06", "Comuna 07", "Comuna 08", "Comuna 09"],  
                            value =["Comuna 1", "Comuna 2", "Comuna 3", "Comuna 4", "Comuna 5", "Comuna 6", "Comuna 7", "Comuna 8", "Comuna 9"], inplace=True)
# En esta línea se reemplazan las comunas con nombre de municipio por el nombre del municipio
df3.COMUNA.replace(to_replace =["Comuna Dagua", "Comuna Jamundi", "Comuna La Cumbre", "Comuna Vijes", "Comuna Yumbo"],  
                            value =["Dagua", "Jamundi", "La Cumbre", "Vijes", "Yumbo"], inplace=True) 
# En esta línea se reemplazan las ciudades diferentes a la jurisdicción de la CC por Cali para facilitar el análisis
df3.CIUDAD.replace(to_replace =["Bogota", "Palmira", "Piendamo", "Calima Darien", "Sevilla"],  
                            value ="Cali", inplace=True)
df3.groupby('CIUDAD')['MATRICULA'].nunique()

<p> Se crea el dicccionario de datos con las variables correspondientes para cruzar con la base de datos limpia de Registro Mercantil. Estos diccionario fueron creados por el grupo de Estudios Económico de la Cámara de Comercio de Cali evidenciando las diferentes maneras de declarar los entes jurídicos, barrios y CIIU por parte de los empresarios. No obstante, este ejercicio deberá ser retroalimentado constantemente para enriquecer el análisis de los datos </p>

In [ ]:
#PN Y PJ
dict_PNPJ = {
    'NOM_ENTE_JURIDICO': ["Persona Natural                                                                           ",	"Sociedad por Acciones Simplificada                                                        ",	"Sociedad Limitada                                                                         ",	"Sociedad en Comandita Simple                                                              ",	"Empresa Unipersonal                                                                       ",	"Sociedad Anónima                                                                          ",	"Sociedad en Comandita por Acciones                                                        ",	"Sociedades Extranjeras                                                                    ",	"Empresa Asociativa de Trabajo                                                             ",	"Sociedades Civiles                                                                        ",	"Sociedad Colectiva                                                                        ",	"Fondos de Pensiones                                                                       ",
],

        'PN_PJ_CORR': [	"Persona Natural",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",	"Persona Juridica",
],
        }

In [ ]:
# Se crea el dataframe con el diccionario de datos referente a los PN y PJ para el posterior cruce de información
df_PNPJ = pd.DataFrame(dict_PNPJ, columns = ["NOM_ENTE_JURIDICO",	"PN_PJ_CORR"])

In [ ]:
# Se unen las dos bases de datos para obtener la información de PN y PJ corregidos
df3 = pd.merge(df3,df_PNPJ, how='left', on=['NOM_ENTE_JURIDICO', 'NOM_ENTE_JURIDICO'])

In [ ]:
# Se procede a graficar las distribuciones según el tratamiento (limpia y sin limpiar)
fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="sum", y=df.MATRICULA_RENOVACION, name="Sin limpiar"))
fig.add_trace(go.Histogram(histfunc="count", y=df3.MATRICULA_RENOVACION , name="limpia"))
fig.update_layout(
    title_text='Número total de registros por Matricula o Renovación', # title of plot
    xaxis_title_text='Número de registros', # xaxis label
    yaxis_title_text=None, # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

# Además se agrega un gráfico de distribución según Ente Juridíco
fig2 = go.Figure(px.histogram(df3, x='MATRICULA_RENOVACION', color='DESC_CATEGORIA', title='Número total de registros por Matricula o Renovación según Ente Jurídico', color_discrete_sequence=px.colors.qualitative.Alphabet))


fig.show()
fig2.show()

<p> Para agregar nuevos valores al diccionario, se recomienda trabajar con el archivo txt para hacer las pruebas. Finalmente, se recomienda actualizar el diccionario agregando en cada campo el valor correspondiente, es decir, para cada actualización se debe contar con el Barrio registrado en la base de datos (BARRIO_REG), el nombre del barrio corregido (NOM_BAR) y el código del barrio en el shapefile del IDESC (COD_BAR). Se agregan como string al final de cada apartado para facilitar el manejo de los datos.

In [ ]:
#BARRIOS

dict_barrios = {
    'BARRIO_REG': ["La Legua","Palermo","Terron Colorado","Vista Hermosa","Sector Patio Bonito","Aguacatal","Alto Aguacatal","Bajo Aguacatal-Las Colinas","Santa Rita","Santa Teresita","Arboledas","Normandia","Juanambu","Centenario","Granada","Versalles","San Vicente","Prados Del Norte (N Sn Vic.)","La Flora","Area Libre-Parque Del Amor","La Campina","La Paz","La Paz (Cabecera)","El Bosque","Menga","Ciudad Los Alamos","Chipichape","Brisas De Los Alamos","Urbanizacion La Merced","Vipasa","Urbanizacion La Flora","Altos De Menga","Altos De Normandia","Altos Santa Monica","El Filo","Golondrinas (Cabecera)","La Maria","San Isidro","Sector Altos Normandia-Bataclan","El Nacional","El Penon","Acueducto San Antonio","San Antonio","San Cayetano","Los Libertadores","San Juan Bosco","Santa Rosa","La Merced","San Pascual","El Calvario","San Pedro","San Nicolas","El Hoyo","El Piloto","Jorge Isaacs","Santander","Porvenir","San Francisco","Las Delicias","Manzanares","Salomia","Fatima","Ignacio Rengifo","Guillermo Valencia","La Isla","Marco Fidel Suarez","Evaristo Garcia","La Esmeralda","Bolivariano","Olaya Herrera","Unidad Res.Bueno Madrid","Flora Industrial","Calima","Sector Calima-La 14","Industria De Licores","La Alianza","El Sena","Los Andes","Los Guayacanes","Chiminangos Segunda Etapa","Chiminangos Primera Etapa","Metropolitano Del Norte","Villa Del Sol","Paseo De Los Almendros","Torres De Comfandi","San Luis","Jorge Eliecer Gaitan","Paso Del Comercio (El Carm)","Los Alcazares","Petecuy Primera Etapa","Petecuy Segunda Etapa","La Rivera Primera Etapa","Los Guaduales","Petecuy Tercera Etapa","Ciudadela Floralia","Fonaviemcali","San Luis Ii","Urbanizacion Calimio","Sector Puente Del Comercio","Alfonso Lopez 1 Etapa","Alfonso Lopez 2 Etapa","Alfonso Lopez 3 Etapa","Puerto Nuevo","Puerto Mallarino","Urb.Angel Del Hogar (A.San)","Siete De Agosto","Los Pinos","San Marino","Las Ceibas","Base Aerea","Parque De La Cana","Fepicol","Primitivo Crespo","Simon Bolivar","Saavedra Galindo","Rafael Uribe Uribe","Uribe Uribe","Santa Monica Popular","La Floresta","Benjamin Herrera","Municipal","Industrial","El Troncal","Las Americas","Atanasio Girardot","Santa Fe","Chapinero","Villa Colombia","El Trebol","La Base","Planta De Tratam.Y Taller","Alameda","Bretana","Junin","Santa Anita La Selva","Guayaquil","Aranjuez","Manuel Maria Buenaventura","Santa Monica Belalcazar","Belalcazar","Sucre","Barrio Obrero","El Dorado","El Guabal","La Libertad","Santa Elena","Las Acacias","Santo Domingo","Jorge Zawadsky","Olimpico","Cristobal Colon","La Selva","Barrio Departamental","Pasoancho","Panamericano","Colseguros Andes","San Cristobal","Las Granjas","San Judas Tadeo","San Judas Tadeo I","San Judas Tadeo Ii","Barrio San Carlos","Maracaibo","Ciudad Modelo","La Independencia","La Esperanza","Urbanizacion Boyaca","El Jardin","La Fortaleza","El Recuerdo","Aguablanca","El Prado","20 De Julio","Prados De Oriente","Villa Del Sur","Jose Holguin Garces","Leon Xiii","Jose Maria Cordoba","San Pedro Claver","Los Conquistadores","La Gran Colombia","San Benito","Primavera","Villanueva","Asturias","Eduardo Santos","Barrio Alfonso Barberena A.","El Paraiso","Fenalco Kennedy","Nueva Floresta Del Ict","Julio Rincon","Doce De Octubre","El Rodeo","Sindical","Bello Horizonte","Ulpiano Lloreda","El Vergel","El Poblado I","El Poblado Ii","Los Comuneros Ii Etapa","Ricardo Balcazar","Omar Torrijos","El Diamante","Lleras Restrepo","Villa Del Lago","Los Robles","Rodrigo Lara Bonilla","Charco Azul","Villablanca","Calipso","Calypso","Yira Castro","Lleras","Lleras Cinta Larga","Marroquin Iii","Los Lagos","Sector Laguna Del Pondaje","El Pondaje","Alfonso Bonilla Aragon","Alirio Mora Beltran","Manuela Beltran","Las Orquideas","Marroquin Ii Etapa","Marroquin I Etapa","Puerta Del Sol","Los Naranjos I","Promociones Populares  B","Los Naranjos Ii","El Retiro","Los Comuneros I Etapa","Laureano Gomez","El Vallado","Vallado","Bajos Ciudad Cordoba","Ciudad Cordoba","Mojica","Mariano Ramos","Republica De Israel","Union De Vivienda Popular","Antonio Narino","Brisas Del Limonar","Ciudad 2000","La Alborado","La Playa","Primero De Mayo","Ciudadela Comfandi","Ciudad Universitaria","Bifamiliares El Dorado","Urbanizacion Caney","Lili","El Ingenio","Las Quintas De Don Simon","Ciudad Capri","La Hacienda","Canaverales Los Samanes","El Limonar","Bosques Del Limonar","El Gran Limonar Cataya","El Gran Limonar","Unicentro Cali","Ciudadela Pasoancho","Prados Del Limonar","Urbanizacion San Joaquin","Buenos Aires","Barrio Caldas","Camilo Torres Restrepo","Los Chorros","Batallon Pichincha","Melendez","Los Farallones","Francisco Eladio Ramirez","Prados Del Sur","Horizontes","Mario Correa Rengifo","Lourdes","Colinas Del Sur","Alferez Real","Napoles","El Jordan","Cuarteles Napoles","Sector Alto De Los Chorros","Alto Melendez-Polvorines","Sector Melendez","Sector Alto Jordan","Alto Napoles","El Refugio","La Cascada","El Lido","Urbanizacion Tequendama","Barrio Eucaristico","U.D.Panamericana-Ant.Hipodromo","U.Dep.Panamericana","San Fernando Nuevo","Urbanizacion Nueva Granada","Santa Isabel","Bellavista","San Fernando Viejo","Miraflores","3 De Julio","El Cedro","Champagnat","Urbanizacion Colseguros","Los Cambulos","El Mortinal","Urbanizacion Militar","Cuarto De Legua Guadalupe","Cuarto De Lugua-Guadalupe","Nueva Tequendama","Camino Real J.Borrero S.","Camino Real-Joaquin Borrero S.","Santa Barbara","Unid.Res.Santiago De Cali","Camino Real Los Fundadores","Camino Real-Los Fundadores","Canaveralejo Seguros Patria","Canaveral","Pampa Linda","Sector Canaveralejo Guadalupe","Sector Bosque Municipal","El Cortijo","Belisario Caicedo","Sector Canaveralejo Sur La Sirena","Siloe","Lleras Camargo","Belen","Urbanizacion Santa Helena","Brisas De Mayo","Tierra Blanca","Pueblo Joven","Cementerio -Carabineros","Parcelacion Monaco La Sultana","Pizamos I","Pizamos Ii","Calimio Desepaz","El Remanso","Los Lideres","Desepaz Invicali","Compartir","Ciudad Talanga","Potrero Grande","Ciudadela Del Rio - Cvc","Valle Grande","Planta De Tratamiento","Urbanizacion Ciudad Jadin","Urbanizacion Ciudad Jardin","Parcelaciones Pance","Urbanizacion Rio Lili","Ciudad Campestre","Club Campestre","Bella Vista","Bolivar","Brisas Del Rio","Barrio Dagua","Cascajal","Cauca Seco","Club Canasgordas","Comfenalco Sede Recreativa","El Boqueron","El Homiguero ( Cabecera)","Hacienda Las Vegas","Parcelacion Andalucia","Parcelaciones El Hormiguero","Piedra Grande","El Saladito (Cabecera)","San Miguel","San Pablo","Cabanas De Carvajal","Felidia (Cabecera)","Las Nieves","Santa Helena","Fray Pena","Barrio Jamundi","Juan Pablo Ii","Alto De Los Mangos","Alto Del Cerro","Anchicaya","Bella Suiza","Campo Alegre","Campoalegre","El Crucero","El Rosario","La Buitrera (Cabecera)","La Luisa","La Riverita","La Sirena","Las Palmas-El Portento-Girasoles","Los Cerros","Parcelaciones Altos Cerros","Parcelaciones Bajos Cerros","Pueblo Nuevo","Sector La Luisa","Sector La Luisa-La Sirena","Sector Las Neblinas","La Castilla (Cabecera)","Las Palmas","Los Limones","Montanitas","La Cumbre","Kilometro 18 (Via Al Mar)","La Elvira (Cabecera)","La Leonera (Cabecera)","La Nueva Estancia","Bajo Cristo Rey-Brisas De Los Cristales","Cerro De Cristo Rey","El Cabuyal","El Mameyal","Los Andes (Cabecera)","Los Andes Parte Baja","Los Mangos","Monaco","Los Andes B-La Riviera-El Saman","Los Parques-Barranquilla","Los Portales Nuevo Rey","Las Vegas","Mayapan Las Vegas","Cinturon  Ecologico","Madre Vieja Rio Cauca","Navarro (Cabecera)","Navarro-La Chanca","Nuevo Orizonte","Area Libre Rios Pance-Jamundi","Cementerio Metropolitano Del Sur","Comfandi Sede Recreativa","Corredor Cali-Jamundi","El Banqueo","El Porvenir","El Trueno","La Castellana","La Voragine","Pance (Cabecera)","Parcelacion Chorro De Plata","Parcelaciones Cerros Pance","Parcelaciones Pance Zona Plana","Panorama","Penas Blancas","Pichinde (Cabecera)","Pizamos Iii - Las Dalias","Popular","Puerto Isaac","San Jorge","Santa Monica","Sect.Asprosocial-Diamante","Sultana-Berlin","Tejares Cristales","Trinidad","Trinidad I","U.Dep.A.Galindo Plaza Toros","Urbanizacion Canaveralejo","Venezuela-Urb. Canaveralejo","Barrio Vijes","Villa Del Prado-El Guabito","La Fonda","Los Mangos (Parte Baja)","Villacarmelo (Cabecera)","Villamercedes I-Villa Luz-Las Garzas","Barrio Yumbo","Imviyumbo San Jorge","ACOPI","FELIDIA","GOLONDRINAS","HORMIGUERO","LA BUITRERA","LA CASTILLA","LA ELVIRA","LA LEONERA","MONTE BELLO","MONTEBELLO","MONTEBELLO (CABECERA)","NAVARRO","PANCE","PICHINDE","SALADITO","PIZAMOS"],

        'NOM_BAR': ["Terrón Colorado",	"Terrón Colorado",	"Terrón Colorado",	"Vista Hermosa",	"Sector Patio Bonito",	"Aguacatal",	"Aguacatal",	"Aguacatal",	"Santa Rita",	"Santa Teresita",	"Arboledas",	"Normandía",	"Juanambú",	"Centenario",	"Granada",	"Versalles",	"San Vicente",	"Prados del Norte",	"La Flora",	"La Flora",	"La Campiña",	"La Paz",	"La Paz",	"El Bosque",	"Menga",	"Ciudad Los Alamos",	"Chipichape",	"Brisas de los Alamos",	"Urbanización La Merced",	"Vipasa",	"Urbanización La Flora",	"Altos de Menga",	"Golondrinas",	"Golondrinas",	"Golondrinas",	"Golondrinas",	"Golondrinas",	"Golondrinas",	"Golondrinas",	"El Nacional",	"El Peñón",	"San Antonio",	"San Antonio",	"San Cayetano",	"Los Libertadores",	"San Juan Bosco",	"Santa Rosa",	"La Merced",	"San Pascual",	"El Calvario",	"San Pedro",	"San Nicolas",	"El Hoyo",	"El Piloto",	"Jorge Isaacs",	"Santander",	"Porvenir",	"Porvenir",	"Las Delicias",	"Manzanares",	"Salomia",	"Fátima",	"Ignacio Rengifo",	"Guillermo Valencia",	"La Isla",	"Marco Fidel Suárez",	"Evaristo García",	"La Esmeralda",	"Bolivariano",	"Olaya Herrera",	"Unidad Residencial Bueno Madrid",	"Flora Industrial",	"Calima",	"Calima",	"Industria de Licores",	"La Alianza",	"El Sena",	"Los Andes",	"Los Guayacanes",	"Chiminangos Segunda Etapa",	"Chiminangos Primera Etapa",	"Metropolitano del Norte",	"Villa del Sol",	"Paseo de Los Almendros",	"Torres de Comfandi",	"San Luís",	"Jorge Eliecer Gaitán",	"Paso del Comercio",	"Los Alcázares",	"Petecuy Primera Etapa",	"Petecuy Segunda Etapa",	"La Rivera Primera Etapa",	"Los Guaduales",	"Petecuy Tercera Etapa",	"Ciudadela Floralia",	"Fonaviemcali",	"San Luís II",	"Urbanización Calimio",	"Sector Puente del Comercio",	"Alfonso López P. 1a. Etapa",	"Alfonso López P. 2a. Etapa",	"Alfonso López P. 3a. Etapa",	"Puerto Nuevo",	"Puerto Mallarino",	"Urbanización El Angel del Hogar",	"Siete de Agosto",	"Los Pinos",	"San Marino",	"Las Ceibas",	"Base Aérea",	"Parque de la Caña",	"Fepicol",	"Primitivo Crespo",	"Simón Bolívar",	"Saavedra Galindo",	"Uribe Uribe",	"Uribe Uribe",	"Santa Mónica Popular",	"La Floresta",	"Benjamín Herrera",	"Municipal",	"Industrial",	"El Troncal",	"Las Américas",	"Atanasio Girardot",	"Santa Fe",	"Chapinero",	"Villa Colombia",	"EL Trébol",	"La Base",	"La Base",	"Alameda",	"Bretaña",	"Junín",	"Junín",	"Guayaquil",	"Aranjuez",	"Manuel María Buenaventura",	"Santa Mónica Belalcázar",	"Belalcázar",	"Sucre",	"Barrio Obrero",	"El Dorado",	"El Guabal",	"La Libertad",	"Santa Elena",	"Las Acacias",	"Santo Domingo",	"Jorge Zawadsky",	"Olímpico",	"Cristóbal Colón",	"La Selva",	"Barrio Departamental",	"Pasoancho",	"Panamericano",	"Colseguros Andes",	"San Cristobal",	"Las Granjas",	"San Judas Tadeo I Etapa",	"San Judas Tadeo I Etapa",	"San Judas Tadeo II Etapa",	"San Carlos",	"Maracaibo",	"La Independencia",	"La Independencia",	"La Esperanza",	"Urbanización Boyacá",	"El Jardín",	"La Fortaleza",	"El Recuerdo",	"Aguablanca",	"El Prado",	"20 de Julio",	"Prados de Oriente",	"Villa del Sur",	"José Holguín Garcés",	"León XIII",	"José María Cordoba",	"San Pedro Claver",	"Los Conquistadores",	"La Gran Colombia",	"San Benito",	"Primavera",	"Villanueva",	"Asturias",	"Eduardo Santos",	"Alfonso Barberena A.",	"El Paraiso",	"Fenalco Kennedy",	"Nueva Floresta",	"Julio Rincón",	"Doce de Octubre",	"El Rodeo",	"Sindical",	"Bello Horizonte",	"Ulpiano Lloreda",	"El Vergel",	"El Poblado I",	"El Poblado II",	"Los Comuneros Segunda Etapa",	"Ricardo Balcázar",	"Omar Torrijos",	"El Diamante",	"Lleras Restrepo",	"Villa del Lago",	"Los Robles",	"Rodrigo Lara Bonilla",	"Charco Azul",	"Villablanca",	"Calipso",	"Calipso",	"Yira Castro",	"Lleras Restrepo II Etapa",	"Lleras Restrepo II Etapa",	"Marroquín III",	"Los Lagos",	"Sector Laguna del Pondaje",	"El Pondaje",	"Alfonso Bonilla Aragón",	"Alirio Mora Beltrán",	"Manuela Beltrán",	"Las Orquídeas",	"Marroquín II Etapa",	"Marroquín I Etapa",	"Puerta del Sol",	"Los Naranjos I",	"Promociones Populares B",	"Los Naranjos II",	"El Retiro",	"Comuneros I",	"Laureano Gómez",	"El Vallado",	"El Vallado",	"Ciudad Cordoba",	"Ciudad Cordoba",	"Mojica",	"Mariano Ramos",	"República de Israel",	"Unión de Vivienda Popular",	"Antonio Nariño",	"Brisas del Limonar",	"Ciudad 2000",	"La Alborada",	"La Playa",	"Primero De Mayo",	"Ciudadela Comfandi",	"Ciudad Universitaria",	"Caney",	"Caney",	"Lili",	"El Ingenio",	"Las Quintas de Don Simón",	"Ciudad Capri",	"La Hacienda",	"Cañaverales - Los Samanes",	"El Limonar",	"Bosques del Limonar",	"El Gran Limonar - Cataya",	"El Gran Limonar",	"Unicentro Cali",	"Ciudadela Pasoancho",	"Prados del Limonar",	"Urbanización San Joaquin",	"Buenos Aires",	"Barrio Caldas",	"Barrio Caldas",	"Los Chorros",	"Meléndez",	"Meléndez",	"Los Farallones",	"Francisco Eladio Ramirez",	"Prados del Sur",	"Horizontes",	"Mario Correa Rengifo",	"Lourdes",	"Colinas del Sur",	"Alférez Real",	"Nápoles",	"El Jordán",	"Cuarteles Napoles",	"Sector Alto de Los Chorros",	"Polvorines",	"Sector Meléndez",	"Sector Alto Jordán",	"Alto Nápoles ",	"El Refugio",	"La Cascada",	"El Lido",	"Urbanización Tequendama",	"Barrio Eucarístico",	"Barrio Eucarístico",	"Barrio Eucarístico",	"San Fernando Nuevo",	"Urbanización Nueva Granada",	"Santa Isabel",	"Bellavista",	"San Fernando Viejo",	"Miraflores",	"3 de Julio",	"El Cedro",	"Champagñat",	"Urbanización Colseguros",	"Los Cambulos",	"El Mortiñal",	"Urbanización Militar",	"Cuarto de Legua - Guadalupe",	"Cuarto de Legua - Guadalupe",	"Nueva Tequendama ",	"Camino Real - Joaquin Borrero Sinisterra",	"Camino Real - Joaquin Borrero Sinisterra",	"Santa Barbara",	"Unidad Residencial Santiago de Cali",	"Cañaveralejo - Seguros Patria",	"Cañaveralejo - Seguros Patria",	"Cañaveralejo - Seguros Patria",	"Cañaveral",	"Pampa Linda",	"Sector Cañaveralejo Guadalupe",	"Sector Bosque Municipal",	"El Cortijo",	"Belisario Caicedo",	"Belisario Caicedo",	"Siloé",	"Lleras Camargo",	"Belén",	"Belén",	"Brisas de Mayo",	"Tierra Blanca",	"Pueblo Joven",	"Carabineros",	"La Sultana",	"Pízamos I",	"Pízamos II",	"Calimio Desepaz",	"El Remanso",	"Los Líderes",	"Desepaz Invicali",	"Compartir",	"Ciudad Talanga",	"Potrero Grande",	"Ciudadela del Río",	"Valle Grande",	"Planta de Tratamiento",	"Urbanización Ciudad Jardín",	"Urbanización Ciudad Jardín",	"Parcelaciones Pance",	"Urbanización Río Lili",	"Ciudad Campestre",	"Club Campestre",	"Altos de Santa Isabel -  La Morelia",	"Bolívar",	"Brisas Del Río",	"Dagua",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Saladito",	"El Saladito",	"El Saladito",	"Felidia",	"Felidia",	"Felidia",	"Felidia",	"Fray Pena",	"Jamundi",	"Juan Pablo II",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Castilla",	"La Castilla",	"La Castilla",	"La Castilla",	"La Cumbre",	"La Elvira",	"La Elvira",	"La Leonera",	"La Nueva Estancia",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes B - La Riviera",	"Los Parques - Barranquilla",	"Los Portales - Nuevo Rey",	"Mayapan - Las Vegas",	"Mayapan - Las Vegas",	"Navarro",	"Navarro",	"Navarro",	"Navarro - La Chanca",	"Nuevo Horizonte",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Panorama",	"Pichinde",	"Pichinde",	"Pízamos III - Las Dalias",	"Popular",	"Puerto Isaac",	"San Jorge",	"Santa Mónica Popular",	"Sector Asprosocial-Diamante",	"Sultana - Berlín - San Francisco ",	"Tejares - Cristales",	"Trinidad I",	"Trinidad I",	"U.D. Alberto Galindo -  Plaza de Toros",	"Venezuela - Urbanización Cañaveralejo",	"Venezuela - Urbanización Cañaveralejo",	"Vijes",	"Villa del Prado - El Guabito",	"Villacarmelo",	"Villacarmelo",	"Villacarmelo",	"Villamercedes I - Villa Luz - Las Garzas",	"Yumbo",	"Yumbo",	"Acopi",	"Felidia",	"Golondrinas",	"Hormiguero",	"La Buitrera",	"La Castilla",	"La Elvira",	"La Leonera",	"Monte Bello",	"Montebello",	"Montebello",	"Navarro",	"Pance",	"Pichinde",	"Saladito","Pízamos I",],

        'COD_BAR': ["0101",	"0101",	"0101",	"0102",	"0196",	"0199",	"0199",	"0199",	"0201",	"0202",	"0203",	"0204",	"0205",	"0206",	"0207",	"0208",	"0209",	"0211",	"0212",	"0212",	"0213",	"0214",	"0214",	"0215",	"0216",	"0217",	"0218",	"0219",	"0293",	"0294",	"0295",	"0296",	"0297",	"0297",	"0297",	"0297",	"0297",	"0297",	"0297",	"0301",	"0302",	"0303",	"0303",	"0304",	"0305",	"0306",	"0307",	"0308",	"0309",	"0310",	"0311",	"0312",	"0313",	"0314",	"0401",	"0402",	"0403",	"0403",	"0404",	"0405",	"0406",	"0407",	"0411",	"0412",	"0413",	"0414",	"0415",	"0416",	"0417",	"0418",	"0419",	"0420",	"0421",	"0421",	"0423",	"0497",	"0501",	"0502",	"0503",	"0504",	"0505",	"0506",	"0595",	"0596",	"0598",	"0601",	"0602",	"0603",	"0604",	"0605",	"0606",	"0607",	"0608",	"0609",	"0610",	"0695",	"0696",	"0697",	"0698",	"0701",	"0702",	"0703",	"0704",	"0705",	"0706",	"0707",	"0708",	"0709",	"0710",	"0711",	"0797",	"0798",	"0801",	"0802",	"0803",	"0804",	"0804",	"0805",	"0806",	"0807",	"0808",	"0809",	"0810",	"0811",	"0812",	"0813",	"0814",	"0815",	"0816",	"0817",	"0817",	"0901",	"0902",	"0903",	"0903",	"0904",	"0905",	"0906",	"0907",	"0908",	"0909",	"0910",	"1001",	"1002",	"1003",	"1004",	"1005",	"1006",	"1007",	"1008",	"1009",	"1010",	"1011",	"1012",	"1013",	"1014",	"1015",	"1016",	"1017",	"1017",	"1018",	"1101",	"1102",	"1103",	"1103",	"1104",	"1105",	"1106",	"1107",	"1108",	"1109",	"1110",	"1111",	"1112",	"1114",	"1115",	"1116",	"1117",	"1118",	"1119",	"1120",	"1121",	"1122",	"1201",	"1202",	"1203",	"1204",	"1205",	"1206",	"1207",	"1208",	"1209",	"1210",	"1211",	"1212",	"1301",	"1302",	"1303",	"1304",	"1305",	"1306",	"1307",	"1308",	"1309",	"1310",	"1311",	"1312",	"1313",	"1314",	"1315",	"1315",	"1390",	"1393",	"1393",	"1394",	"1395",	"1397",	"1398",	"1401",	"1402",	"1403",	"1404",	"1405",	"1406",	"1495",	"1496",	"1498",	"1499",	"1501",	"1502",	"1503",	"1504",	"1504",	"1596",	"1596",	"1598",	"1601",	"1602",	"1603",	"1604",	"1605",	"1697",	"1698",	"1701",	"1702",	"1703",	"1705",	"1774",	"1774",	"1775",	"1780",	"1782",	"1783",	"1784",	"1786",	"1787",	"1788",	"1789",	"1790",	"1791",	"1793",	"1794",	"1796",	"1801",	"1802",	"1802",	"1803",	"1804",	"1804",	"1805",	"1807",	"1808",	"1809",	"1810",	"1811",	"1812",	"1813",	"1814",	"1815",	"1816",	"1890",	"1891",	"1896",	"1897",	"1898",	"1901",	"1902",	"1903",	"1904",	"1905",	"1905",	"1905",	"1906",	"1907",	"1908",	"1909",	"1910",	"1911",	"1912",	"1913",	"1914",	"1915",	"1916",	"1917",	"1918",	"1919",	"1919",	"1921",	"1922",	"1922",	"1982",	"1984",	"1988",	"1988",	"1988",	"1992",	"1994",	"1995",	"1997",	"2001",	"2002",	"2002",	"2003",	"2004",	"2005",	"2005",	"2006",	"2007",	"2008",	"2097",	"2099",	"2101",	"2102",	"2103",	"2104",	"2105",	"2106",	"2107",	"2108",	"2196",	"2197",	"2198",	"2199",	"2201",	"2201",	"2296",	"2297",	"2298",	"2299",	"1981",	"Bolívar",	"Brisas Del Río",	"Dagua",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Hormiguero",	"El Saladito",	"El Saladito",	"El Saladito",	"Felidia",	"Felidia",	"Felidia",	"Felidia",	"Fray Pena",	"Jamundi",	"Juan Pablo II",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Buitrera",	"La Castilla",	"La Castilla",	"La Castilla",	"La Castilla",	"La Cumbre",	"La Elvira",	"La Elvira",	"La Leonera",	"La Nueva Estancia",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes - Corregimiento",	"Los Andes  Corregimiento",	"Los Andes  Corregimiento",	"Los Andes  Corregimiento",	"Los Andes  Corregimiento",	"Los Andes  Corregimiento",	"0597",	"0594",	"1785",	"1781",	"1781",	"Navarro",	"Navarro",	"Navarro",	"0315",	"Nuevo Horizonte",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Pance",	"Panorama",	"Pichinde",	"Pichinde",	"2195",	"0410",	"Puerto Isaac",	"San Jorge",	"0805",	"1399",	"0408",	"1983",	"Trinidad I",	"Trinidad I",	"1999",	"2098",	"2098",	"Vijes",	"0599",	"Villacarmelo",	"Villacarmelo",	"Villacarmelo",	"2194",	"Yumbo",	"Yumbo",	"Acopi",	"Felidia",	"Golondrinas",	"Hormiguero",	"La Buitrera",	"La Castilla",	"La Elvira",	"La Leonera",	"Monte Bello",	"Montebello",	"Montebello",	"Navarro",	"Pance",	"Pichinde",	"Saladito","2101"],
        }

    

In [ ]:
# Se crea el dataframe con el diccionario de datos referente a los nombre de los barrios para el posterior cruce de información
df_barrios = pd.DataFrame(dict_barrios, columns = ['BARRIO_REG', 'NOM_BAR', 'COD_BAR'])
df.to_excel(r'ESALES_CCC_ENE_V3(FEB_08_2022_EEE3).xlsx', index = False)



In [ ]:
# Para el cruce de las bases de datos se hace la identificación de valores unicos en la columna de barrios de la base de datos de Registro Mercantil limpia
#pd.set_option('display.max_rows', None)
#df3.groupby('BARRIO')['MATRICULA'].nunique()

In [ ]:
# Para unir las bases es necesario volver las llaves en mayuscula en ambos dataframes
df3['BARRIO'] = df3['BARRIO'].str.upper()
df_barrios['BARRIO_REG'] = df_barrios['BARRIO_REG'].str.upper()

In [ ]:
# Se renombra la variable a cruzar para que tenga el mismo nombre en las dos bases
df_barrios.rename(columns={'BARRIO_REG':'BARRIO'}, inplace=True)

In [ ]:
# Se eliminan los espacio en blanco de las llaves para evitar errores al cruzar las bases de datos
df3.BARRIO = df3.BARRIO.str.replace(' ', '')
df_barrios.BARRIO = df_barrios.BARRIO.str.replace(' ', '')

In [ ]:
# Al eliminar los espacios se pueden generar llaves duplicadas, lo cual genera filas duplicadas al realizar un left join, por tal motivo, se eliminan las llaves duplicadas del diccionario
duplicados_df_barrios = df_barrios[df_barrios.duplicated(['BARRIO'], keep='last')]
print("El número de llaves duplicadas es:", duplicados_df_barrios.BARRIO.count(), sep='\n')

In [ ]:
# Se eliminan las llaves duplicadas del diccionrio de Barrios
df_barrios=df_barrios.sort_values('BARRIO').drop_duplicates('BARRIO', keep='last')

In [ ]:
# Se unen las dos bases de datos para obtener la información de barrios corregidos
df3.BARRIO = df3.BARRIO.astype(str)
df_barrios.BARRIO = df_barrios.BARRIO.astype(str)
df3 = pd.merge(df3, df_barrios, how='left', on=['BARRIO','BARRIO'])

In [ ]:
#CIIU

dict_CIIU = {
	'CODIGO_CIIU': ["12100",	"722000",	"851202",	"111",	"112",	"113",	"114",	"119",	"121",	"122",	"123",	"124",	"125",	"126",	"127",	"128",	"129",	"130",	"141",	"142",	"143",	"144",	"145",	"149",	"150",	"161",	"162",	"163",	"164",	"210",	"220",	"240",	"311",	"312",	"321",	"322",	"510",	"710",	"722",	"729",	"811",	"812",	"820",	"899",	"910",	"990",	"1011",	"1012",	"1020",	"1030",	"1040",	"1051",	"1052",	"1061",	"1062",	"1063",	"1071",	"1072",	"1081",	"1082",	"1083",	"1084",	"1089",	"1090",	"1101",	"1102",	"1103",	"1104",	"1200",	"1311",	"1312",	"1313",	"1391",	"1392",	"1393",	"1394",	"1399",	"1410",	"1420",	"1430",	"1511",	"1512",	"1513",	"1521",	"1522",	"1523",	"1610",	"1620",	"1630",	"1640",	"1690",	"1701",	"1702",	"1709",	"1811",	"1812",	"1820",	"1910",	"1921",	"1922",	"2011",	"2012",	"2013",	"2014",	"2021",	"2022",	"2023",	"2029",	"2030",	"2100",	"2211",	"2212",	"2219",	"2221",	"2229",	"2310",	"2391",	"2392",	"2393",	"2394",	"2395",	"2396",	"2399",	"2410",	"2421",	"2429",	"2431",	"2432",	"2511",	"2512",	"2513",	"2520",	"2591",	"2592",	"2593",	"2599",	"2610",	"2620",	"2630",	"2640",	"2651",	"2652",	"2660",	"2670",	"2680",	"2711",	"2712",	"2720",	"2731",	"2732",	"2740",	"2750",	"2790",	"2811",	"2812",	"2813",	"2814",	"2815",	"2816",	"2817",	"2818",	"2819",	"2821",	"2822",	"2823",	"2824",	"2825",	"2826",	"2829",	"2910",	"2920",	"2930",	"3011",	"3012",	"3020",	"3030",	"3040",	"3091",	"3092",	"3099",	"3110",	"3120",	"3210",	"3220",	"3230",	"3240",	"3250",	"3290",	"3311",	"3312",	"3313",	"3314",	"3315",	"3319",	"3320",	"3511",	"3512",	"3513",	"3514",	"3520",	"3530",	"3600",	"3700",	"3811",	"3812",	"3821",	"3822",	"3830",	"3900",	"4111",	"4112",	"4210",	"4220",	"4290",	"4311",	"4312",	"4321",	"4322",	"4329",	"4330",	"4390",	"4511",	"4512",	"4520",	"4530",	"4541",	"4542",	"4610",	"4620",	"4631",	"4632",	"4641",	"4642",	"4643",	"4644",	"4645",	"4649",	"4651",	"4652",	"4653",	"4659",	"4661",	"4662",	"4663",	"4664",	"4665",	"4669",	"4690",	"4711",	"4719",	"4721",	"4722",	"4723",	"4724",	"4729",	"4731",	"4732",	"4741",	"4742",	"4751",	"4752",	"4753",	"4754",	"4755",	"4759",	"4761",	"4762",	"4769",	"4771",	"4772",	"4773",	"4774",	"4775",	"4781",	"4782",	"4789",	"4791",	"4792",	"4799",	"4911",	"4912",	"4921",	"4922",	"4923",	"4930",	"5011",	"5012",	"5021",	"5022",	"5111",	"5112",	"5121",	"5122",	"5210",	"5221",	"5222",	"5223",	"5224",	"5229",	"5310",	"5320",	"5511",	"5512",	"5513",	"5514",	"5519",	"5520",	"5530",	"5590",	"5611",	"5612",	"5613",	"5619",	"5621",	"5629",	"5630",	"5811",	"5812",	"5813",	"5819",	"5820",	"5911",	"5912",	"5913",	"5914",	"5920",	"6010",	"6020",	"6110",	"6120",	"6130",	"6190",	"6201",	"6202",	"6209",	"6311",	"6312",	"6391",	"6399",	"6411",	"6412",	"6421",	"6422",	"6423",	"6424",	"6431",	"6432",	"6491",	"6492",	"6493",	"6494",	"6495",	"6499",	"6511",	"6512",	"6513",	"6514",	"6521",	"6522",	"6531",	"6532",	"6611",	"6612",	"6613",	"6614",	"6615",	"6619",	"6621",	"6629",	"6630",	"6810",	"6820",	"6910",	"6920",	"7010",	"7020",	"7110",	"7120",	"7210",	"7220",	"7310",	"7320",	"7410",	"7420",	"7490",	"7500",	"7710",	"7721",	"7722",	"7729",	"7730",	"7740",	"7810",	"7820",	"7830",	"7911",	"7912",	"7990",	"8010",	"8020",	"8030",	"8110",	"8121",	"8129",	"8130",	"8211",	"8219",	"8220",	"8230",	"8291",	"8292",	"8299",	"8411",	"8412",	"8413",	"8414",	"8415",	"8421",	"8422",	"8423",	"8424",	"8430",	"8511",	"8512",	"8513",	"8521",	"8522",	"8523",	"8530",	"8541",	"8542",	"8543",	"8544",	"8551",	"8552",	"8553",	"8559",	"8560",	"8610",	"8621",	"8622",	"8691",	"8692",	"8699",	"8710",	"8720",	"8730",	"8790",	"8810",	"8890",	"9001",	"9002",	"9003",	"9004",	"9005",	"9006",	"9007",	"9008",	"9101",	"9102",	"9103",	"9200",	"9311",	"9312",	"9319",	"9321",	"9329",	"9411",	"9412",	"9420",	"9491",	"9492",	"9499",	"9511",	"9512",	"9521",	"9522",	"9523",	"9524",	"9529",	"9601",	"9602",	"9603",	"9609",	"9700",	"9810",	"9820",	"9900",	"0111",	"0112",	"0113",	"0114",	"0115",	"0119",	"0121",	"0122",	"0123",	"0124",	"0125",	"0126",	"0127",	"0128",	"0129",	"0130",	"0141",	"0142",	"0143",	"0144",	"0145",	"0149",	"0150",	"0161",	"0162",	"0163",	"0164",	"0170",	"0210",	"0220",	"0230",	"0240",	"0311",	"0312",	"0321",	"0322",	"0510",	"0520",	"0610",	"0620",	"0710",	"0721",	"0722",	"0723",	"0729",	"0811",	"0812",	"0820",	"0891",	"0892",	"0899",	"0910",	"0990", "7111",	"7112", "891"],

	'CIIU_COR': ["4723",	"8299",	"8692",	"0111",	"0112",	"0113",	"0114",	"0119",	"0121",	"0122",	"0123",	"0124",	"0125",	"0126",	"0127",	"0128",	"0129",	"0130",	"0141",	"0142",	"0143",	"0144",	"0145",	"0149",	"0150",	"0161",	"0162",	"0163",	"0164",	"0210",	"0220",	"0240",	"0311",	"0312",	"0321",	"0322",	"0510",	"0710",	"0722",	"0729",	"0811",	"0812",	"0820",	"0899",	"0910",	"0990",	"1011",	"1012",	"1020",	"1030",	"1040",	"1051",	"1052",	"1061",	"1062",	"1063",	"1071",	"1072",	"1081",	"1082",	"1083",	"1084",	"1089",	"1090",	"1101",	"1102",	"1103",	"1104",	"1200",	"1311",	"1312",	"1313",	"1391",	"1392",	"1393",	"1394",	"1399",	"1410",	"1420",	"1430",	"1511",	"1512",	"1513",	"1521",	"1522",	"1523",	"1610",	"1620",	"1630",	"1640",	"1690",	"1701",	"1702",	"1709",	"1811",	"1812",	"1820",	"1910",	"1921",	"1922",	"2011",	"2012",	"2013",	"2014",	"2021",	"2022",	"2023",	"2029",	"2030",	"2100",	"2211",	"2212",	"2219",	"2221",	"2229",	"2310",	"2391",	"2392",	"2393",	"2394",	"2395",	"2396",	"2399",	"2410",	"2421",	"2429",	"2431",	"2432",	"2511",	"2512",	"2513",	"2520",	"2591",	"2592",	"2593",	"2599",	"2610",	"2620",	"2630",	"2640",	"2651",	"2652",	"2660",	"2670",	"2680",	"2711",	"2712",	"2720",	"2731",	"2732",	"2740",	"2750",	"2790",	"2811",	"2812",	"2813",	"2814",	"2815",	"2816",	"2817",	"2818",	"2819",	"2821",	"2822",	"2823",	"2824",	"2825",	"2826",	"2829",	"2910",	"2920",	"2930",	"3011",	"3012",	"3020",	"3030",	"3040",	"3091",	"3092",	"3099",	"3110",	"3120",	"3210",	"3220",	"3230",	"3240",	"3250",	"3290",	"3311",	"3312",	"3313",	"3314",	"3315",	"3319",	"3320",	"3511",	"3512",	"3513",	"3514",	"3520",	"3530",	"3600",	"3700",	"3811",	"3812",	"3821",	"3822",	"3830",	"3900",	"4111",	"4112",	"4210",	"4220",	"4290",	"4311",	"4312",	"4321",	"4322",	"4329",	"4330",	"4390",	"4511",	"4512",	"4520",	"4530",	"4541",	"4542",	"4610",	"4620",	"4631",	"4632",	"4641",	"4642",	"4643",	"4644",	"4645",	"4649",	"4651",	"4652",	"4653",	"4659",	"4661",	"4662",	"4663",	"4664",	"4665",	"4669",	"4690",	"4711",	"4719",	"4721",	"4722",	"4723",	"4724",	"4729",	"4731",	"4732",	"4741",	"4742",	"4751",	"4752",	"4753",	"4754",	"4755",	"4759",	"4761",	"4762",	"4769",	"4771",	"4772",	"4773",	"4774",	"4775",	"4781",	"4782",	"4789",	"4791",	"4792",	"4799",	"4911",	"4912",	"4921",	"4922",	"4923",	"4930",	"5011",	"5012",	"5021",	"5022",	"5111",	"5112",	"5121",	"5122",	"5210",	"5221",	"5222",	"5223",	"5224",	"5229",	"5310",	"5320",	"5511",	"5512",	"5513",	"5514",	"5519",	"5520",	"5530",	"5590",	"5611",	"5612",	"5613",	"5619",	"5621",	"5629",	"5630",	"5811",	"5812",	"5813",	"5819",	"5820",	"5911",	"5912",	"5913",	"5914",	"5920",	"6010",	"6020",	"6110",	"6120",	"6130",	"6190",	"6201",	"6202",	"6209",	"6311",	"6312",	"6391",	"6399",	"6411",	"6412",	"6421",	"6422",	"6423",	"6424",	"6431",	"6432",	"6491",	"6492",	"6493",	"6494",	"6495",	"6499",	"6511",	"6512",	"6513",	"6514",	"6521",	"6522",	"6531",	"6532",	"6611",	"6612",	"6613",	"6614",	"6615",	"6619",	"6621",	"6629",	"6630",	"6810",	"6820",	"6910",	"6920",	"7010",	"7020",	"7110",	"7120",	"7210",	"7220",	"7310",	"7320",	"7410",	"7420",	"7490",	"7500",	"7710",	"7721",	"7722",	"7729",	"7730",	"7740",	"7810",	"7820",	"7830",	"7911",	"7912",	"7990",	"8010",	"8020",	"8030",	"8110",	"8121",	"8129",	"8130",	"8211",	"8219",	"8220",	"8230",	"8291",	"8292",	"8299",	"8411",	"8412",	"8413",	"8414",	"8415",	"8421",	"8422",	"8423",	"8424",	"8430",	"8511",	"8512",	"8513",	"8521",	"8522",	"8523",	"8530",	"8541",	"8542",	"8543",	"8544",	"8551",	"8552",	"8553",	"8559",	"8560",	"8610",	"8621",	"8622",	"8691",	"8692",	"8699",	"8710",	"8720",	"8730",	"8790",	"8810",	"8890",	"9001",	"9002",	"9003",	"9004",	"9005",	"9006",	"9007",	"9008",	"9101",	"9102",	"9103",	"9200",	"9311",	"9312",	"9319",	"9321",	"9329",	"9411",	"9412",	"9420",	"9491",	"9492",	"9499",	"9511",	"9512",	"9521",	"9522",	"9523",	"9524",	"9529",	"9601",	"9602",	"9603",	"9609",	"9700",	"9810",	"9820",	"9900",	"0111",	"0112",	"0113",	"0114",	"0115",	"0119",	"0121",	"0122",	"0123",	"0124",	"0125",	"0126",	"0127",	"0128",	"0129",	"0130",	"0141",	"0142",	"0143",	"0144",	"0145",	"0149",	"0150",	"0161",	"0162",	"0163",	"0164",	"0170",	"0210",	"0220",	"0230",	"0240",	"0311",	"0312",	"0321",	"0322",	"0510",	"0520",	"0610",	"0620",	"0710",	"0721",	"0722",	"0723",	"0729",	"0811",	"0812",	"0820",	"0891",	"0892",	"0899",	"0910",	"0990",	"7111",	"7112", "0891"],

	'DESC_CIIU_COR': [	"Comercio al por menor de carnes (incluye aves de corral), productos cárnicos, pescados y productos de mar, en establecimientos especializados",	"Otras actividades de servicio de apoyo a las empresas n.c.p.",	"Actividades de apoyo terapéutico",	"Cultivo de cereales (excepto arroz), legumbres y semillas oleaginosas ",	"Cultivo de arroz ",	"Cultivo de hortalizas, raíces y tubérculos ",	"Cultivo de tabaco ",	"Otros cultivos transitorios n.c.p.",	"Cultivo de frutas tropicales y subtropicales",	"Cultivo de plátano y banano",	"Cultivo de café",	"Cultivo de caña de azúcar",	"Cultivo de flor de corte",	"Cultivo de palma para aceite (palma africana) y otros frutos oleaginosos",	"Cultivo de plantas con las que se preparan bebidas",	"Cultivo de especias y de plantas aromáticas y medicinales ",	"Otros cultivos permanentes n.c.p.",	"Propagación de plantas (actividades de los viveros, excepto viveros forestales) ",	"Cría de ganado bovino y bufalino",	"Cría de caballos y otros equinos ",	"Cría de ovejas y cabras ",	"Cría de ganado porcino",	"Cría de aves de corral",	"Cría de otros animales n.c.p.",	"Explotación mixta (agrícola y pecuaria) ",	"Actividades de apoyo a la agricultura ",	"Actividades de apoyo a la ganadería",	"Actividades posteriores a la cosecha ",	"Tratamiento de semillas para propagación ",	"Silvicultura y otras actividades forestales",	"Extracción de madera ",	"Servicios de apoyo a la silvicultura ",	"Pesca marítima ",	"Pesca de agua dulce ",	"Acuicultura marítima ",	"Acuicultura de agua dulce",	"Extracción de hulla (carbón de piedra)",	"Extracción de minerales de hierro",	"Extracción de oro y otros metales preciosos",	"Extracción de otros minerales metalíferos no ferrosos n.c.p.",	"Extracción de piedra, arena, arcillas comunes, yeso y anhidrita",	"Extracción de arcillas de uso industrial, caliza, caolín y bentonitas",	"Extracción de esmeraldas, piedras preciosas y semipreciosas",	"Extracción de otros minerales no metálicos n.c.p.",	"Actividades de apoyo para la extracción de petróleo y de gas natural",	"Actividades de apoyo para otras actividades de explotación de minas y canteras",	"Procesamiento y conservación de carne y productos cárnicos",	"Procesamiento y conservación de pescados, crustáceos y moluscos",	"Procesamiento y conservación de frutas, legumbres, hortalizas y tubérculos",	"Elaboración de aceites y grasas de origen vegetal y animal",	"Elaboración de productos lácteos",	"Elaboración de productos de molinería",	"Elaboración de almidones y productos derivados del almidón",	"Trilla de café",	"Descafeinado, tostión y molienda del café",	"Otros derivados del café",	"Elaboración y refinación de azúcar",	"Elaboración de panela",	"Elaboración de productos de panadería",	"Elaboración de cacao, chocolate y productos de confitería",	"Elaboración de macarrones, fideos, alcuzcuz y productos farináceos similares",	"Elaboración de comidas y platos preparados",	"Elaboración de otros productos alimenticios n.c.p.",	"Elaboración de alimentos preparados para animales",	"Destilación, rectificación y mezcla de bebidas alcohólicas",	"Elaboración de bebidas fermentadas no destiladas",	"Producción de malta, elaboración de cervezas y otras bebidas malteadas",	"Elaboración de bebidas no alcohólicas, producción de aguas minerales y de otras aguas embotelladas",	"Elaboración de productos de tabaco",	"Preparación e hilatura de fibras textiles",	"Tejeduría de productos textiles",	"Acabado de productos textiles",	"Fabricación de tejidos de punto y ganchillo",	"Confección de artículos con materiales textiles, excepto prendas de vestir",	"Fabricación de tapetes y alfombras para pisos",	"Fabricación de cuerdas, cordeles, cables, bramantes y redes",	"Fabricación de otros artículos textiles n.c.p.",	"Confección de prendas de vestir, excepto prendas de piel",	"Fabricación de artículos de piel",	"Fabricación de artículos de punto y ganchillo",	"Curtido y recurtido de cueros; recurtido y teñido de pieles",	"Fabricación de artículos de viaje, bolsos de mano y artículos similares elaborados en cuero, y fabricación de artículos de talabartería y guarnicionería",	"Fabricación de artículos de viaje, bolsos de mano y artículos similares; artículos de talabartería y guarnicionería elaborados en otros materiales",	"Fabricación de calzado de cuero y piel, con cualquier tipo de suela",	"Fabricación de otros tipos de calzado, excepto calzado de cuero y piel",	"Fabricación de partes del calzado",	"Aserrado, acepillado e impregnación de la madera",	"Fabricación de hojas de madera para enchapado; fabricación de tableros contrachapados, tableros laminados, tableros de partículas y otros tableros y paneles",	"Fabricación de partes y piezas de madera, de carpintería y ebanistería para la construcción",	"Fabricación de recipientes de madera",	"Fabricación de otros productos de madera; fabricación de artículos de corcho, cestería y espartería",	"Fabricación de pulpas (pastas) celulósicas; papel y cartón",	"Fabricación de papel y cartón ondulado (corrugado); fabricación de envases, empaques y de embalajes de papel y cartón.",	"Fabricación de otros artículos de papel y cartón",	"Actividades de impresión",	"Actividades de servicios relacionados con la impresión",	"Producción de copias a partir de grabaciones originales ",	"Fabricación de productos de hornos de coque",	"Fabricación de productos de la refinación del petróleo",	"Actividad de mezcla de combustibles",	"Fabricación de sustancias y productos químicos básicos",	"Fabricación de abonos y compuestos inorgánicos nitrogenados",	"Fabricación de plásticos en formas primarias",	"Fabricación de caucho sintético en formas primarias",	"Fabricación de plaguicidas y otros productos químicos de uso agropecuario",	"Fabricación de pinturas, barnices y revestimientos similares, tintas para impresión y masillas",	"Fabricación de jabones y detergentes, preparados para limpiar y pulir; perfumes y preparados de tocador",	"Fabricación de otros productos químicos n.c.p.",	"Fabricación de fibras sintéticas y artificiales",	"Fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos de uso farmacéutico",	"Fabricación de llantas y neumáticos de caucho",	"Reencauche de llantas usadas",	"Fabricación de formas básicas de caucho y otros productos de caucho n.c.p.",	"Fabricación de formas básicas de plástico",	"Fabricación de artículos de plástico n.c.p.",	"Fabricación de vidrio y productos de vidrio",	"Fabricación de productos refractarios",	"Fabricación de materiales de arcilla para la construcción",	"Fabricación de otros productos de cerámica y porcelana",	"Fabricación de cemento, cal y yeso",	"Fabricación de artículos de hormigón, cemento y yeso",	"Corte, tallado y acabado de la piedra",	"Fabricación de otros productos minerales no metálicos n.c.p.",	"Industrias básicas de hierro y de acero",	"Industrias básicas de metales preciosos",	"Industrias básicas de otros metales no ferrosos",	"Fundición de hierro y de acero",	"Fundición de metales no ferrosos ",	"Fabricación de productos metálicos para uso estructural",	"Fabricación de tanques, depósitos y recipientes de metal, excepto los utilizados para el envase o transporte de mercancías",	"Fabricación de generadores de vapor, excepto calderas de agua caliente para calefacción central",	"Fabricación de armas y municiones",	"Forja, prensado, estampado y laminado de metal; pulvimetalurgia",	"Tratamiento y revestimiento de metales; mecanizado",	"Fabricación de artículos de cuchillería, herramientas de mano y artículos de ferretería",	"Fabricación de otros productos elaborados de metal n.c.p.",	"Fabricación de componentes y tableros electrónicos",	"Fabricación de computadoras y de equipo periférico",	"Fabricación de equipos de comunicación",	"Fabricación de aparatos electrónicos de consumo",	"Fabricación de equipo de medición, prueba, navegación y control",	"Fabricación de relojes",	"Fabricación de equipo de irradiación y equipo electrónico de uso médico y terapéutico",	"Fabricación de instrumentos ópticos y equipo fotográfico",	"Fabricación de medios magnéticos y ópticos para almacenamiento de datos",	"Fabricación de motores, generadores y transformadores eléctricos",	"Fabricación de aparatos de distribución y control de la energía eléctrica",	"Fabricación de pilas, baterías y acumuladores eléctricos",	"Fabricación de hilos y cables eléctricos y de fibra óptica",	"Fabricación de dispositivos de cableado",	"Fabricación de equipos eléctricos de iluminación",	"Fabricación de aparatos de uso doméstico",	"Fabricación de otros tipos de equipo eléctrico n.c.p.",	"Fabricación de motores, turbinas, y partes para motores de combustión interna",	"Fabricación de equipos de potencia hidráulica y neumática",	"Fabricación de otras bombas, compresores, grifos y válvulas",	"Fabricación de cojinetes, engranajes, trenes de engranajes y piezas de transmisión",	"Fabricación de hornos, hogares y quemadores industriales",	"Fabricación de equipo de elevación y manipulación",	"Fabricación de maquinaria y equipo de oficina (excepto computadoras y equipo periférico)",	"Fabricación de herramientas manuales con motor",	"Fabricación de otros tipos de maquinaria y equipo de uso general n.c.p.",	"Fabricación de maquinaria agropecuaria y forestal",	"Fabricación de máquinas formadoras de metal y de máquinas herramienta",	"Fabricación de maquinaria para la metalurgia",	"Fabricación de maquinaria para explotación de minas y canteras y para obras de construcción",	"Fabricación de maquinaria para la elaboración de alimentos, bebidas y tabaco",	"Fabricación de maquinaria para la elaboración de productos textiles, prendas de vestir y cueros",	"Fabricación de otros tipos de maquinaria y equipo de uso especial n.c.p.",	"Fabricación de vehículos automotores y sus motores",	"Fabricación de carrocerías para vehículos automotores; fabricación de remolques y semirremolques ",	"Fabricación de partes, piezas (autopartes) y accesorios (lujos) para vehículos automotores",	"Construcción de barcos y de estructuras flotantes",	"Construcción de embarcaciones de recreo y deporte",	"Fabricación de locomotoras y de material rodante para ferrocarriles",	"Fabricación de aeronaves, naves espaciales y de maquinaria conexa",	"Fabricación de vehículos militares de combate",	"Fabricación de motocicletas",	"Fabricación de bicicletas y de sillas de ruedas para personas con discapacidad",	"Fabricación de otros tipos de equipo de transporte n.c.p.",	"Fabricación de muebles ",	"Fabricación de colchones y somieres",	"Fabricación de joyas, bisutería y artículos conexos",	"Fabricación de instrumentos musicales",	"Fabricación de artículos y equipo para la práctica del deporte",	"Fabricación de juegos, juguetes y rompecabezas",	"Fabricación de instrumentos, aparatos y materiales médicos y odontológicos (incluido mobiliario)",	"Otras industrias manufactureras n.c.p.",	"Mantenimiento y reparación especializado de productos elaborados en metal",	"Mantenimiento y reparación especializado de maquinaria y equipo",	"Mantenimiento y reparación especializado de equipo electrónico y óptico",	"Mantenimiento y reparación especializado de equipo eléctrico",	"Mantenimiento y reparación especializado de equipo de transporte, excepto los vehículos automotores, motocicletas y bicicletas",	"Mantenimiento y reparación de otros tipos de equipos y sus componentes n.c.p.",	"Instalación especializada de maquinaria y equipo industrial ",	"Generación de energía eléctrica",	"Transmisión de energía eléctrica",	"Distribución de energía eléctrica",	"Comercialización de energía eléctrica",	"Producción de gas; distribución de combustibles gaseosos por tuberías",	"Suministro de vapor y aire acondicionado",	"Captación, tratamiento y distribución de agua",	"Evacuación y tratamiento de aguas residuales",	"Recolección de desechos no peligrosos",	"Recolección de desechos peligrosos",	"Tratamiento y disposición de desechos no peligrosos",	"Tratamiento y disposición de desechos peligrosos",	"Recuperación de materiales",	"Actividades de saneamiento ambiental y otros servicios de gestión de desechos",	"Construcción de edificios residenciales",	"Construcción de edificios no residenciales",	"Construcción de carreteras y vías de ferrocarril",	"Construcción de proyectos de servicio público",	"Construcción de otras obras de ingeniería civil",	"Demolición",	"Preparación del terreno",	"Instalaciones eléctricas",	"Instalaciones de fontanería, calefacción y aire acondicionado",	"Otras instalaciones especializadas",	"Terminación y acabado de edificios y obras de ingeniería civil",	"Otras actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Comercio de vehículos automotores nuevos",	"Comercio de vehículos automotores usados",	"Mantenimiento y reparación de vehículos automotores",	"Comercio de partes, piezas (autopartes) y accesorios (lujos) para vehículos automotores",	"Comercio de motocicletas y de sus partes, piezas y accesorios",	"Mantenimiento y reparación de motocicletas y de sus partes y piezas",	"Comercio al por mayor a cambio de una retribución o por contrata",	"Comercio al por mayor de materias primas agropecuarias; animales vivos",	"Comercio al por mayor de productos alimenticios",	"Comercio al por mayor de bebidas y tabaco",	"Comercio al por mayor de productos textiles, productos confeccionados para uso doméstico",	"Comercio al por mayor de prendas de vestir",	"Comercio al por mayor de calzado",	"Comercio al por mayor de aparatos y equipo de uso doméstico",	"Comercio al por mayor de productos farmacéuticos, medicinales, cosméticos y de tocador",	"Comercio al por mayor de otros utensilios domésticos n.c.p.",	"Comercio al por mayor de computadores, equipo periférico y programas de informática",	"Comercio al por mayor de equipo, partes y piezas electrónicos y de telecomunicaciones",	"Comercio al por mayor de maquinaria y equipo agropecuarios",	"Comercio al por mayor de otros tipos de maquinaria y equipo n.c.p.",	"Comercio al por mayor de combustibles sólidos, líquidos, gaseosos y productos conexos",	"Comercio al por mayor de metales y productos metalíferos",	"Comercio al por mayor de materiales de construcción, artículos de ferretería, pinturas, productos de vidrio, equipo y materiales de fontanería y calefacción",	"Comercio al por mayor de productos químicos básicos, cauchos y plásticos en formas primarias y productos químicos de uso agropecuario",	"Comercio al por mayor de desperdicios, desechos y chatarra",	"Comercio al por mayor de otros productos n.c.p.",	"Comercio al por mayor no especializado",	"Comercio al por menor en establecimientos no especializados con surtido compuesto principalmente por alimentos, bebidas o tabaco",	"Comercio al por menor en establecimientos no especializados, con surtido compuesto principalmente por productos diferentes de alimentos (víveres en general), bebidas y tabaco",	"Comercio al por menor de productos agrícolas para el consumo en establecimientos especializados",	"Comercio al por menor de leche, productos lácteos y huevos, en establecimientos especializados",	"Comercio al por menor de carnes (incluye aves de corral), productos cárnicos, pescados y productos de mar, en establecimientos especializados",	"Comercio al por menor de bebidas y productos del tabaco, en establecimientos especializados",	"Comercio al por menor de otros productos alimenticios n.c.p., en establecimientos especializados",	"Comercio al por menor de combustible para automotores",	"Comercio al por menor de lubricantes (aceites, grasas), aditivos y productos de limpieza para vehículos automotores",	"Comercio al por menor de computadores, equipos periféricos, programas de informática y equipos de telecomunicaciones en establecimientos especializados",	"Comercio al por menor de equipos y aparatos de sonido y de video, en establecimientos especializados",	"Comercio al por menor de productos textiles en establecimientos especializados",	"Comercio al por menor de artículos de ferretería, pinturas y productos de vidrio en establecimientos especializados",	"Comercio al por menor de tapices, alfombras y cubrimientos para paredes y pisos en establecimientos especializados",	"Comercio al por menor de electrodomésticos y gasodomésticos de uso doméstico, muebles y equipos de iluminación",	"Comercio al por menor de artículos y utensilios de uso doméstico",	"Comercio al por menor de otros artículos domésticos en establecimientos especializados",	"Comercio al por menor de libros, periódicos, materiales y artículos de papelería y escritorio, en establecimientos especializados",	"Comercio al por menor de artículos deportivos, en establecimientos especializados ",	"Comercio al por menor de otros artículos culturales y de entretenimiento n.c.p. en establecimientos especializados",	"Comercio al por menor de prendas de vestir y sus accesorios (incluye artículos de piel) en establecimientos especializados",	"Comercio al por menor de todo tipo de calzado y artículos de cuero y sucedáneos del cuero en establecimientos especializados.",	"Comercio al por menor de productos farmacéuticos y medicinales, cosméticos y artículos de tocador en establecimientos especializados",	"Comercio al por menor de otros productos nuevos en establecimientos especializados",	"Comercio al por menor de artículos de segunda mano",	"Comercio al por menor de alimentos, bebidas y tabaco, en puestos de venta móviles",	"Comercio al por menor de productos textiles, prendas de vestir y calzado, en puestos de venta móviles",	"Comercio al por menor de otros productos en puestos de venta móviles",	"Comercio al por menor realizado a través de Internet",	"Comercio al por menor realizado a través de casas de venta o por correo",	"Otros tipos de comercio al por menor no realizado en establecimientos, puestos de venta o mercados.",	"Transporte férreo de pasajeros",	"Transporte férreo de carga ",	"Transporte de pasajeros",	"Transporte mixto",	"Transporte de carga por carretera",	"Transporte por tuberías",	"Transporte de pasajeros marítimo y de cabotaje ",	"Transporte de carga marítimo y de cabotaje ",	"Transporte fluvial de pasajeros",	"Transporte fluvial de carga",	"Transporte aéreo nacional de pasajeros ",	"Transporte aéreo internacional de pasajeros ",	"Transporte aéreo nacional de carga ",	"Transporte aéreo internacional de carga ",	"Almacenamiento y depósito",	"Actividades de estaciones, vías y servicios complementarios para el transporte terrestre",	"Actividades de puertos y servicios complementarios para el transporte acuático",	"Actividades de aeropuertos, servicios de navegación aérea y demás actividades conexas al transporte aéreo",	"Manipulación de carga",	"Otras actividades complementarias al transporte",	"Actividades postales nacionales",	"Actividades de mensajería",	"Alojamiento en hoteles ",	"Alojamiento en apartahoteles",	"Alojamiento en centros vacacionales ",	"Alojamiento rural",	"Otros tipos de alojamientos para visitantes",	"Actividades de zonas de camping y parques para vehículos recreacionales",	"Servicio por horas ",	"Otros tipos de alojamiento n.c.p.",	"Expendio a la mesa de comidas preparadas",	"Expendio por autoservicio de comidas preparadas",	"Expendio de comidas preparadas en cafeterías",	"Otros tipos de expendio de comidas preparadas n.c.p.",	"Catering para eventos",	"Actividades de otros servicios de comidas",	"Expendio de bebidas alcohólicas para el consumo dentro del establecimiento",	"Edición de libros",	"Edición de directorios y listas de correo",	"Edición de periódicos, revistas y otras publicaciones periódicas",	"Otros trabajos de edición",	"Edición de programas de informática (software)",	"Actividades de producción de películas cinematográficas, videos, programas, anuncios y comerciales de televisión",	"Actividades de posproducción de películas cinematográficas, videos, programas, anuncios y comerciales de televisión",	"Actividades de distribución de películas cinematográficas, videos, programas, anuncios y comerciales de televisión",	"Actividades de exhibición de películas cinematográficas y videos",	"Actividades de grabación de sonido y edición de música",	"Actividades de programación y transmisión en el servicio de radiodifusión sonora",	"Actividades de programación y transmisión de televisión",	"Actividades de telecomunicaciones alámbricas",	"Actividades de telecomunicaciones inalámbricas",	"Actividades de telecomunicación satelital",	"Otras actividades de telecomunicaciones",	"Actividades de desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas)",	"Actividades de consultoría informática y actividades de administración de instalaciones informáticas",	"Otras actividades de tecnologías de información y actividades de servicios informáticos",	"Procesamiento de datos, alojamiento (hosting) y actividades relacionadas",	"Portales web",	"Actividades de agencias de noticias",	"Otras actividades de servicio de información n.c.p.",	"Banco Central",	"Bancos comerciales",	"Actividades de las corporaciones financieras",	"Actividades de las compañías de financiamiento",	"Banca de segundo piso",	"Actividades de las cooperativas financieras",	"Fideicomisos, fondos y entidades financieras similares",	"Fondos de cesantías",	"Leasing financiero (arrendamiento financiero)",	"Actividades financieras de fondos de empleados y otras formas asociativas del sector solidario",	"Actividades de compra de cartera o factoring",	"Otras actividades de distribución de fondos",	"Instituciones especiales oficiales",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones n.c.p.",	"Seguros generales ",	"Seguros de vida",	"Reaseguros",	"Capitalización",	"Servicios de seguros sociales de salud",	"Servicios de seguros sociales de riesgos profesionales",	"Régimen de prima media con prestación definida (RPM)",	"Régimen de ahorro individual (RAI)",	"Administración de mercados financieros",	"Corretaje de valores y de contratos de productos básicos",	"Otras actividades relacionadas con el mercado de valores",	"Actividades de las casas de cambio",	"Actividades de los profesionales de compra y venta de divisas",	"Otras actividades auxiliares de las actividades de servicios financieros n.c.p.",	"Actividades de agentes y corredores de seguros",	"Evaluación de riesgos y daños, y otras actividades de servicios auxiliares",	"Actividades de administración de fondos",	"Actividades inmobiliarias realizadas con bienes propios o arrendados",	"Actividades inmobiliarias realizadas a cambio de una retribución o por contrata ",	"Actividades jurídicas",	"Actividades de contabilidad, teneduría de libros, auditoría financiera y asesoría tributaria",	"Actividades de administración empresarial",	"Actividades de consultaría de gestión",	"Actividades de arquitectura e ingeniería y otras actividades conexas de consultoría técnica",	"Ensayos y análisis técnicos",	"Investigaciones y desarrollo experimental en el campo de las ciencias naturales y la ingeniería ",	"Investigaciones y desarrollo experimental en el campo de las ciencias sociales y las humanidades",	"Publicidad",	"Estudios de mercado y realización de encuestas de opinión pública",	"Actividades especializadas de diseño ",	"Actividades de fotografía",	"Otras actividades profesionales, científicas y técnicas n.c.p.",	"Actividades veterinarias",	"Alquiler y arrendamiento de vehículos automotores",	"Alquiler y arrendamiento de equipo recreativo y deportivo",	"Alquiler de videos y discos ",	"Alquiler y arrendamiento de otros efectos personales y enseres domésticos n.c.p.",	"Alquiler y arrendamiento de otros tipos de maquinaria, equipo y bienes tangibles n.c.p.",	"Arrendamiento de propiedad intelectual y productos similares, excepto obras protegidas por derechos de autor",	"Actividades de agencias de empleo",	"Actividades de agencias de empleo temporal",	"Otras actividades de suministro de recurso humano",	"Actividades de las agencias de viaje",	"Actividades de operadores turísticos",	"Otros servicios de reserva y actividades relacionadas",	"Actividades de seguridad privada",	"Actividades de servicios de sistemas de seguridad",	"Actividades de detectives e investigadores privados",	"Actividades combinadas de apoyo a instalaciones",	"Limpieza general interior de edificios",	"Otras actividades de limpieza de edificios e instalaciones industriales",	"Actividades de paisajismo y servicios de mantenimiento conexos",	"Actividades combinadas de servicios administrativos de oficina",	"Fotocopiado, preparación de documentos y otras actividades especializadas de apoyo a oficina",	"Actividades de centros de llamadas (Call center)",	"Organización de convenciones y eventos comerciales",	"Actividades de agencias de cobranza y oficinas de calificación crediticia",	"Actividades de envase y empaque",	"Otras actividades de servicio de apoyo a las empresas n.c.p.",	"Actividades legislativas de la administración pública",	"Actividades ejecutivas de la administración pública",	"Regulación de las actividades de organismos que prestan servicios de salud, educativos, culturales y otros servicios sociales, excepto servicios de seguridad social ",	"Actividades reguladoras y facilitadoras de la actividad económica",	"Actividades de los otros órganos de control",	"Relaciones exteriores ",	"Actividades de defensa",	"Orden público y actividades de seguridad",	"Administración de justicia",	"Actividades de planes de seguridad social de afiliación obligatoria",	"Educación de la primera infancia",	"Educación preescolar",	"Educación básica primaria",	"Educación básica secundaria ",	"Educación media académica",	"Educación media técnica y de formación laboral",	"Establecimientos que combinan diferentes niveles de educación ",	"Educación técnica profesional",	"Educación tecnológica",	"Educación de instituciones universitarias o de escuelas tecnológicas",	"Educación de universidades",	"Formación académica no formal ",	"Enseñanza deportiva y recreativa",	"Enseñanza cultural",	"Otros tipos de educación n.c.p.",	"Actividades de apoyo a la educación",	"Actividades de hospitales y clínicas, con internación",	"Actividades de la práctica médica, sin internación",	"Actividades de la práctica odontológica",	"Actividades de apoyo diagnóstico",	"Actividades de apoyo terapéutico",	"Otras actividades de atención de la salud humana",	"Actividades de atención residencial medicalizada de tipo general",	"Actividades de atención residencial, para el cuidado de pacientes con retardo mental, enfermedad mental y consumo de sustancias psicoactivas",	"Actividades de atención en instituciones para el cuidado de personas mayores y/o discapacitadas",	"Otras actividades de atención en instituciones con alojamiento",	"Actividades de asistencia social sin alojamiento para personas mayores y discapacitadas",	"Otras actividades de asistencia social sin alojamiento",	"Creación literaria",	"Creación musical",	"Creación teatral",	"Creación audiovisual",	"Artes plásticas y visuales",	"Actividades teatrales",	"Actividades de espectáculos musicales en vivo",	"Otras actividades de espectáculos en vivo",	"Actividades de bibliotecas y archivos",	"Actividades y funcionamiento de museos, conservación de edificios y sitios históricos",	"Actividades de jardines botánicos, zoológicos y reservas naturales",	"Actividades de juegos de azar y apuestas",	"Gestión de instalaciones deportivas",	"Actividades de clubes deportivos",	"Otras actividades deportivas",	"Actividades de parques de atracciones y parques temáticos",	"Otras actividades recreativas y de esparcimiento n.c.p.",	"Actividades de asociaciones empresariales y de empleadores",	"Actividades de asociaciones profesionales",	"Actividades de sindicatos de empleados",	"Actividades de asociaciones religiosas",	"Actividades de asociaciones políticas",	"Actividades de otras asociaciones n.c.p.",	"Mantenimiento y reparación de computadores y de equipo periférico",	"Mantenimiento y reparación de equipos de comunicación",	"Mantenimiento y reparación de aparatos electrónicos de consumo",	"Mantenimiento y reparación de aparatos y equipos domésticos y de jardinería ",	"Reparación de calzado y artículos de cuero",	"Reparación de muebles y accesorios para el hogar",	"Mantenimiento y reparación de otros efectos personales y enseres domésticos",	"Lavado y limpieza, incluso la limpieza en seco, de productos textiles y de piel",	"Peluquería y otros tratamientos de belleza",	"Pompas fúnebres y actividades relacionadas",	"Otras actividades de servicios personales n.c.p.",	"Actividades de los hogares individuales como empleadores de personal doméstico",	"Actividades no diferenciadas de los hogares individuales como productores de bienes para uso propio",	"Actividades no diferenciadas de los hogares individuales como productores de servicios para uso propio",	"Actividades de organizaciones y entidades extraterritoriales",	"Cultivo de cereales (excepto arroz), legumbres y semillas oleaginosas ",	"Cultivo de arroz ",	"Cultivo de hortalizas, raíces y tubérculos ",	"Cultivo de tabaco ",	"Cultivo de plantas textiles ",	"Otros cultivos transitorios n.c.p.",	"Cultivo de frutas tropicales y subtropicales",	"Cultivo de plátano y banano",	"Cultivo de café",	"Cultivo de caña de azúcar",	"Cultivo de flor de corte",	"Cultivo de palma para aceite (palma africana) y otros frutos oleaginosos",	"Cultivo de plantas con las que se preparan bebidas",	"Cultivo de especias y de plantas aromáticas y medicinales ",	"Otros cultivos permanentes n.c.p.",	"Propagación de plantas (actividades de los viveros, excepto viveros forestales) ",	"Cría de ganado bovino y bufalino",	"Cría de caballos y otros equinos ",	"Cría de ovejas y cabras ",	"Cría de ganado porcino",	"Cría de aves de corral",	"Cría de otros animales n.c.p.",	"Explotación mixta (agrícola y pecuaria) ",	"Actividades de apoyo a la agricultura ",	"Actividades de apoyo a la ganadería",	"Actividades posteriores a la cosecha ",	"Tratamiento de semillas para propagación ",	"Caza ordinaria y mediante trampas y actividades de servicios conexas ",	"Silvicultura y otras actividades forestales",	"Extracción de madera ",	"Recolección de productos forestales diferentes a la madera",	"Servicios de apoyo a la silvicultura ",	"Pesca marítima ",	"Pesca de agua dulce ",	"Acuicultura marítima ",	"Acuicultura de agua dulce",	"Extracción de hulla (carbón de piedra)",	"Extracción de carbón lignito",	"Extracción de petróleo crudo",	"Extracción de gas natural",	"Extracción de minerales de hierro",	"Extracción de minerales de uranio y de torio",	"Extracción de oro y otros metales preciosos",	"Extracción de minerales de níquel",	"Extracción de otros minerales metalíferos no ferrosos n.c.p.",	"Extracción de piedra, arena, arcillas comunes, yeso y anhidrita",	"Extracción de arcillas de uso industrial, caliza, caolín y bentonitas",	"Extracción de esmeraldas, piedras preciosas y semipreciosas",	"Extracción de minerales para la fabricación de abonos y productos químicos",	"Extracción de halita (sal)",	"Extracción de otros minerales no metálicos n.c.p.",	"Actividades de apoyo para la extracción de petróleo y de gas natural",	"Actividades de apoyo para otras actividades de explotación de minas y canteras",	"Actividades de arquitectura",	"Actividades de ingeniería y otras actividades conexas de consultoría técnica",	"Extracción de minerales para la fabricación de abonos y productos químicos"],

	'SECCION_COR': [	"G",	"N",	"Q",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"C",	"D",	"D",	"D",	"D",	"D",	"D",	"E",	"E",	"E",	"E",	"E",	"E",	"E",	"E",	"F",	"F",	"F",	"F",	"F",	"F",	"F",	"F",	"F",	"F",	"F",	"F",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"G",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"H",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"I",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"J",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"K",	"L",	"L",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"M",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"N",	"O",	"O",	"O",	"O",	"O",	"O",	"O",	"O",	"O",	"O",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"P",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"Q",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"R",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"S",	"T",	"T",	"T",	"U",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"A",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"B",	"M",	"M",	"B"
],

        'DESC_SECCION_COR': [	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Actividades de servicios administrativos y de apoyo",	"Actividades de atención de la salud humana y de asistencia social",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Industrias manufactureras",	"Suministro de electricidad, gas, vapor y aire acondicionado",	"Suministro de electricidad, gas, vapor y aire acondicionado",	"Suministro de electricidad, gas, vapor y aire acondicionado",	"Suministro de electricidad, gas, vapor y aire acondicionado",	"Suministro de electricidad, gas, vapor y aire acondicionado",	"Suministro de electricidad, gas, vapor y aire acondicionado",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Construcción",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Transporte y almacenamiento",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Alojamiento y servicios de comida",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Información y comunicaciones",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades financieras y de seguros",	"Actividades inmobiliarias",	"Actividades inmobiliarias",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Actividades de servicios administrativos y de apoyo",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades de atención de la salud humana y de asistencia social",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Actividades artísticas, de entretenimiento y recreación",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Otras actividades de servicios",	"Actividades de los hogares individuales en calidad de empleadores; actividades no diferenciadas de los hogares individuales como productores de",	"Actividades de los hogares individuales en calidad de empleadores; actividades no diferenciadas de los hogares individuales como productores de",	"Actividades de los hogares individuales en calidad de empleadores; actividades no diferenciadas de los hogares individuales como productores de",	"Actividades de organizaciones y entidades extraterritoriales",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Agricultura, ganadería, caza, silvicultura y pesca",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Explotación de minas y canteras",	"Actividades profesionales, científicas y técnicas",	"Actividades profesionales, científicas y técnicas",	"Explotación de minas y canteras"],

	'DIVISION': [	"47",	"82",	"86",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"02",	"02",	"02",	"03",	"03",	"03",	"03",	"05",	"07",	"07",	"07",	"08",	"08",	"08",	"08",	"09",	"09",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"10",	"11",	"11",	"11",	"11",	"12",	"13",	"13",	"13",	"13",	"13",	"13",	"13",	"13",	"14",	"14",	"14",	"15",	"15",	"15",	"15",	"15",	"15",	"16",	"16",	"16",	"16",	"16",	"17",	"17",	"17",	"18",	"18",	"18",	"19",	"19",	"19",	"20",	"20",	"20",	"20",	"20",	"20",	"20",	"20",	"20",	"21",	"22",	"22",	"22",	"22",	"22",	"23",	"23",	"23",	"23",	"23",	"23",	"23",	"23",	"24",	"24",	"24",	"24",	"24",	"25",	"25",	"25",	"25",	"25",	"25",	"25",	"25",	"26",	"26",	"26",	"26",	"26",	"26",	"26",	"26",	"26",	"27",	"27",	"27",	"27",	"27",	"27",	"27",	"27",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"28",	"29",	"29",	"29",	"30",	"30",	"30",	"30",	"30",	"30",	"30",	"30",	"31",	"31",	"32",	"32",	"32",	"32",	"32",	"32",	"33",	"33",	"33",	"33",	"33",	"33",	"33",	"35",	"35",	"35",	"35",	"35",	"35",	"36",	"37",	"38",	"38",	"38",	"38",	"38",	"39",	"41",	"41",	"42",	"42",	"42",	"43",	"43",	"43",	"43",	"43",	"43",	"43",	"45",	"45",	"45",	"45",	"45",	"45",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"46",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"47",	"49",	"49",	"49",	"49",	"49",	"49",	"50",	"50",	"50",	"50",	"51",	"51",	"51",	"51",	"52",	"52",	"52",	"52",	"52",	"52",	"53",	"53",	"55",	"55",	"55",	"55",	"55",	"55",	"55",	"55",	"56",	"56",	"56",	"56",	"56",	"56",	"56",	"58",	"58",	"58",	"58",	"58",	"59",	"59",	"59",	"59",	"59",	"60",	"60",	"61",	"61",	"61",	"61",	"62",	"62",	"62",	"63",	"63",	"63",	"63",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"64",	"65",	"65",	"65",	"65",	"65",	"65",	"65",	"65",	"66",	"66",	"66",	"66",	"66",	"66",	"66",	"66",	"66",	"68",	"68",	"69",	"69",	"70",	"70",	"71",	"71",	"72",	"72",	"73",	"73",	"74",	"74",	"74",	"75",	"77",	"77",	"77",	"77",	"77",	"77",	"78",	"78",	"78",	"79",	"79",	"79",	"80",	"80",	"80",	"81",	"81",	"81",	"81",	"82",	"82",	"82",	"82",	"82",	"82",	"82",	"84",	"84",	"84",	"84",	"84",	"84",	"84",	"84",	"84",	"84",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"85",	"86",	"86",	"86",	"86",	"86",	"86",	"87",	"87",	"87",	"87",	"88",	"88",	"90",	"90",	"90",	"90",	"90",	"90",	"90",	"90",	"91",	"91",	"91",	"92",	"93",	"93",	"93",	"93",	"93",	"94",	"94",	"94",	"94",	"94",	"94",	"95",	"95",	"95",	"95",	"95",	"95",	"95",	"96",	"96",	"96",	"96",	"97",	"98",	"98",	"99",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"01",	"02",	"02",	"02",	"02",	"03",	"03",	"03",	"03",	"05",	"05",	"06",	"06",	"07",	"07",	"07",	"07",	"07",	"08",	"08",	"08",	"08",	"08",	"08",	"09",	"09",	"71",	"71",	"08"],

        'DES_DIVI': [	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades de atención de la salud humana",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Silvicultura y extracción de madera",	"Silvicultura y extracción de madera",	"Silvicultura y extracción de madera",	"Pesca y acuicultura",	"Pesca y acuicultura",	"Pesca y acuicultura",	"Pesca y acuicultura",	"Extracción de carbón de piedra y lignito",	"Extracción de minerales metalíferos",	"Extracción de minerales metalíferos",	"Extracción de minerales metalíferos",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Actividades de servicios de apoyo para la explotación de minas ",	"Actividades de servicios de apoyo para la explotación de minas ",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de productos alimenticios",	"Elaboración de bebidas",	"Elaboración de bebidas",	"Elaboración de bebidas",	"Elaboración de bebidas",	"Elaboración de productos de tabaco",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Fabricación de productos textiles",	"Confección de prendas de vestir",	"Confección de prendas de vestir",	"Confección de prendas de vestir",	"Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y ",	"Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y ",	"Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y ",	"Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y ",	"Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y ",	"Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y ",	"Transformación de la madera y fabricación de productos de madera y de corcho, excepto muebles; fabricación de artículos de cestería y espartería",	"Transformación de la madera y fabricación de productos de madera y de corcho, excepto muebles; fabricación de artículos de cestería y espartería",	"Transformación de la madera y fabricación de productos de madera y de corcho, excepto muebles; fabricación de artículos de cestería y espartería",	"Transformación de la madera y fabricación de productos de madera y de corcho, excepto muebles; fabricación de artículos de cestería y espartería",	"Transformación de la madera y fabricación de productos de madera y de corcho, excepto muebles; fabricación de artículos de cestería y espartería",	"Fabricación de papel, cartón y productos de papel y cartón",	"Fabricación de papel, cartón y productos de papel y cartón",	"Fabricación de papel, cartón y productos de papel y cartón",	"Actividades de impresión y de producción de copias a partir de grabaciones originales ",	"Actividades de impresión y de producción de copias a partir de grabaciones originales ",	"Actividades de impresión y de producción de copias a partir de grabaciones originales ",	"Coquización, fabricación de productos de la refinación del petróleo y actividad de mezcla de combustibles ",	"Coquización, fabricación de productos de la refinación del petróleo y actividad de mezcla de combustibles ",	"Coquización, fabricación de productos de la refinación del petróleo y actividad de mezcla de combustibles ",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de sustancias y productos químicos",	"Fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos de uso farmacéutico",	"Fabricación de productos de caucho y de plástico",	"Fabricación de productos de caucho y de plástico",	"Fabricación de productos de caucho y de plástico",	"Fabricación de productos de caucho y de plástico",	"Fabricación de productos de caucho y de plástico",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de otros productos minerales no metálicos",	"Fabricación de productos metalúrgicos básicos",	"Fabricación de productos metalúrgicos básicos",	"Fabricación de productos metalúrgicos básicos",	"Fabricación de productos metalúrgicos básicos",	"Fabricación de productos metalúrgicos básicos",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos elaborados de metal, excepto maquinaria y equipo",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de productos informáticos, electrónicos y ópticos",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de aparatos y equipo eléctrico",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de maquinaria y equipo n.c.p.",	"Fabricación de vehículos automotores, remolques y semirremolques",	"Fabricación de vehículos automotores, remolques y semirremolques",	"Fabricación de vehículos automotores, remolques y semirremolques",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de otros tipos de equipo de transporte",	"Fabricación de muebles, colchones y somieres",	"Fabricación de muebles, colchones y somieres",	"Otras industrias manufactureras",	"Otras industrias manufactureras",	"Otras industrias manufactureras",	"Otras industrias manufactureras",	"Otras industrias manufactureras",	"Otras industrias manufactureras",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Instalación, mantenimiento y reparación especializado de maquinaria y equipo",	"Suministro de electricidad, gas, vapor y aire acondicionado ",	"Suministro de electricidad, gas, vapor y aire acondicionado ",	"Suministro de electricidad, gas, vapor y aire acondicionado ",	"Suministro de electricidad, gas, vapor y aire acondicionado ",	"Suministro de electricidad, gas, vapor y aire acondicionado ",	"Suministro de electricidad, gas, vapor y aire acondicionado ",	"Captación, tratamiento y distribución de agua",	"Evacuación y tratamiento de aguas residuales",	"Recolección, tratamiento y disposición de desechos, recuperación de materiales",	"Recolección, tratamiento y disposición de desechos, recuperación de materiales",	"Recolección, tratamiento y disposición de desechos, recuperación de materiales",	"Recolección, tratamiento y disposición de desechos, recuperación de materiales",	"Recolección, tratamiento y disposición de desechos, recuperación de materiales",	"Actividades de saneamiento ambiental y otros servicios de gestión de desechos",	"Construcción de edificios",	"Construcción de edificios",	"Obras de ingeniería civil",	"Obras de ingeniería civil",	"Obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios",	"Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios",	"Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios",	"Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios",	"Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios",	"Comercio, mantenimiento y reparación de vehículos automotores y motocicletas, sus partes, piezas y accesorios",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por mayor y en comisión o por contrata, excepto el comercio de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Comercio al por menor (incluso el comercio al por menor de combustibles), excepto el de vehículos automotores y motocicletas",	"Transporte terrestre; transporte por tuberías",	"Transporte terrestre; transporte por tuberías",	"Transporte terrestre; transporte por tuberías",	"Transporte terrestre; transporte por tuberías",	"Transporte terrestre; transporte por tuberías",	"Transporte terrestre; transporte por tuberías",	"Transporte acuático",	"Transporte acuático",	"Transporte acuático",	"Transporte acuático",	"Transporte aéreo",	"Transporte aéreo",	"Transporte aéreo",	"Transporte aéreo",	"Almacenamiento y actividades complementarias al transporte",	"Almacenamiento y actividades complementarias al transporte",	"Almacenamiento y actividades complementarias al transporte",	"Almacenamiento y actividades complementarias al transporte",	"Almacenamiento y actividades complementarias al transporte",	"Almacenamiento y actividades complementarias al transporte",	"Correo y servicios de mensajería",	"Correo y servicios de mensajería",	"Alojamiento",	"Alojamiento",	"Alojamiento",	"Alojamiento",	"Alojamiento",	"Alojamiento",	"Alojamiento",	"Alojamiento",	"Actividades de servicios de comidas y bebidas",	"Actividades de servicios de comidas y bebidas",	"Actividades de servicios de comidas y bebidas",	"Actividades de servicios de comidas y bebidas",	"Actividades de servicios de comidas y bebidas",	"Actividades de servicios de comidas y bebidas",	"Actividades de servicios de comidas y bebidas",	"Actividades de edición",	"Actividades de edición",	"Actividades de edición",	"Actividades de edición",	"Actividades de edición",	"Actividades cinematográficas, de video y producción de programas de televisión, grabación de sonido y edición de música",	"Actividades cinematográficas, de video y producción de programas de televisión, grabación de sonido y edición de música",	"Actividades cinematográficas, de video y producción de programas de televisión, grabación de sonido y edición de música",	"Actividades cinematográficas, de video y producción de programas de televisión, grabación de sonido y edición de música",	"Actividades cinematográficas, de video y producción de programas de televisión, grabación de sonido y edición de música",	"Actividades de programación, transmisión y/o difusión",	"Actividades de programación, transmisión y/o difusión",	"Telecomunicaciones",	"Telecomunicaciones",	"Telecomunicaciones",	"Telecomunicaciones",	"Desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas), consultoría informática y actividades relacionadas",	"Desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas), consultoría informática y actividades relacionadas",	"Desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas), consultoría informática y actividades relacionadas",	"Actividades de servicios de información",	"Actividades de servicios de información",	"Actividades de servicios de información",	"Actividades de servicios de información",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Actividades de servicios financieros, excepto las de seguros y de pensiones",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Seguros (incluso el reaseguro), seguros sociales y fondos de pensiones, excepto la seguridad social",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades auxiliares de las actividades de servicios financieros",	"Actividades inmobiliarias",	"Actividades inmobiliarias",	"Actividades jurídicas y de contabilidad",	"Actividades jurídicas y de contabilidad",	"Actividades de administración empresarial; actividades de consultoría de gestión",	"Actividades de administración empresarial; actividades de consultoría de gestión",	"Actividades de arquitectura e ingeniería; ensayos y análisis técnicos",	"Actividades de arquitectura e ingeniería; ensayos y análisis técnicos",	"Investigación científica y desarrollo",	"Investigación científica y desarrollo",	"Publicidad y estudios de mercado",	"Publicidad y estudios de mercado",	"Otras actividades profesionales, científicas y técnicas",	"Otras actividades profesionales, científicas y técnicas",	"Otras actividades profesionales, científicas y técnicas",	"Actividades veterinarias",	"Actividades de alquiler y arrendamiento",	"Actividades de alquiler y arrendamiento",	"Actividades de alquiler y arrendamiento",	"Actividades de alquiler y arrendamiento",	"Actividades de alquiler y arrendamiento",	"Actividades de alquiler y arrendamiento",	"Actividades de empleo",	"Actividades de empleo",	"Actividades de empleo",	"Actividades de las agencias de viajes, operadores turísticos, servicios de reserva y actividades relacionadas",	"Actividades de las agencias de viajes, operadores turísticos, servicios de reserva y actividades relacionadas",	"Actividades de las agencias de viajes, operadores turísticos, servicios de reserva y actividades relacionadas",	"Actividades de seguridad e investigación privada",	"Actividades de seguridad e investigación privada",	"Actividades de seguridad e investigación privada",	"Actividades de servicios a edificios y paisajismo (jardines, zonas verdes)",	"Actividades de servicios a edificios y paisajismo (jardines, zonas verdes)",	"Actividades de servicios a edificios y paisajismo (jardines, zonas verdes)",	"Actividades de servicios a edificios y paisajismo (jardines, zonas verdes)",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Actividades administrativas y de apoyo de oficina y otras actividades de apoyo a las empresas",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Administración pública y defensa; planes de seguridad social de afiliación obligatoria",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Educación",	"Actividades de atención de la salud humana",	"Actividades de atención de la salud humana",	"Actividades de atención de la salud humana",	"Actividades de atención de la salud humana",	"Actividades de atención de la salud humana",	"Actividades de atención de la salud humana",	"Actividades de atención residencial medicalizada",	"Actividades de atención residencial medicalizada",	"Actividades de atención residencial medicalizada",	"Actividades de atención residencial medicalizada",	"Actividades de asistencia social sin alojamiento",	"Actividades de asistencia social sin alojamiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades creativas, artísticas y de entretenimiento",	"Actividades de bibliotecas, archivos, museos y otras actividades culturales",	"Actividades de bibliotecas, archivos, museos y otras actividades culturales",	"Actividades de bibliotecas, archivos, museos y otras actividades culturales",	"Actividades de juegos de azar y apuestas",	"Actividades deportivas y actividades recreativas y de esparcimiento",	"Actividades deportivas y actividades recreativas y de esparcimiento",	"Actividades deportivas y actividades recreativas y de esparcimiento",	"Actividades deportivas y actividades recreativas y de esparcimiento",	"Actividades deportivas y actividades recreativas y de esparcimiento",	"Actividades de asociaciones",	"Actividades de asociaciones",	"Actividades de asociaciones",	"Actividades de asociaciones",	"Actividades de asociaciones",	"Actividades de asociaciones",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Mantenimiento y reparación de computadores, efectos personales y enseres domésticos",	"Otras actividades de servicios personales",	"Otras actividades de servicios personales",	"Otras actividades de servicios personales",	"Otras actividades de servicios personales",	"Actividades de los hogares individuales como empleadores de personal doméstico",	"Actividades no diferenciadas de los hogares individuales como productores de bienes y servicios para uso propio",	"Actividades no diferenciadas de los hogares individuales como productores de bienes y servicios para uso propio",	"Actividades de organizaciones y entidades extraterritoriales",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Agricultura, ganadería, caza y actividades de servicios conexas ",	"Silvicultura y extracción de madera",	"Silvicultura y extracción de madera",	"Silvicultura y extracción de madera",	"Silvicultura y extracción de madera",	"Pesca y acuicultura",	"Pesca y acuicultura",	"Pesca y acuicultura",	"Pesca y acuicultura",	"Extracción de carbón de piedra y lignito",	"Extracción de carbón de piedra y lignito",	"Extracción de petróleo crudo y gas natural",	"Extracción de petróleo crudo y gas natural",	"Extracción de minerales metalíferos",	"Extracción de minerales metalíferos",	"Extracción de minerales metalíferos",	"Extracción de minerales metalíferos",	"Extracción de minerales metalíferos",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Extracción de otras minas y canteras",	"Actividades de servicios de apoyo para la explotación de minas",	"Actividades de servicios de apoyo para la explotación de minas",	"Actividades de arquitectura e ingeniería; ensayos y análisis técnicos",	"Actividades de arquitectura e ingeniería; ensayos y análisis técnicos",	"Extracción de otras minas y canteras"],

        'GRUPO': [	"472",	"829",	"869",	"011",	"011",	"011",	"011",	"011",	"012",	"012",	"012",	"012",	"012",	"012",	"012",	"012",	"012",	"013",	"014",	"014",	"014",	"014",	"014",	"014",	"015",	"016",	"016",	"016",	"016",	"021",	"022",	"024",	"031",	"031",	"032",	"032",	"051",	"071",	"072",	"072",	"081",	"081",	"82",	"089",	"091",	"099",	"101",	"101",	"102",	"103",	"104",	"105",	"105",	"106",	"106",	"106",	"107",	"107",	"108",	"108",	"108",	"108",	"108",	"109",	"110",	"110",	"110",	"110",	"120",	"131",	"131",	"131",	"139",	"139",	"139",	"139",	"139",	"141",	"142",	"143",	"151",	"151",	"151",	"152",	"152",	"152",	"161",	"162",	"163",	"164",	"169",	"170",	"170",	"170",	"181",	"181",	"182",	"191",	"192",	"192",	"201",	"201",	"201",	"201",	"202",	"202",	"202",	"202",	"203",	"210",	"221",	"221",	"221",	"222",	"222",	"231",	"239",	"239",	"239",	"239",	"239",	"239",	"239",	"241",	"242",	"242",	"243",	"243",	"251",	"251",	"251",	"252",	"259",	"259",	"259",	"259",	"261",	"262",	"263",	"264",	"265",	"265",	"266",	"267",	"268",	"271",	"271",	"272",	"273",	"273",	"274",	"275",	"279",	"281",	"281",	"281",	"281",	"281",	"281",	"281",	"281",	"281",	"282",	"282",	"282",	"282",	"282",	"282",	"282",	"291",	"292",	"293",	"301",	"301",	"302",	"303",	"304",	"309",	"309",	"309",	"311",	"312",	"321",	"322",	"323",	"324",	"325",	"329",	"331",	"331",	"331",	"331",	"331",	"331",	"332",	"351",	"351",	"351",	"351",	"352",	"353",	"360",	"370",	"381",	"381",	"382",	"382",	"383",	"390",	"411",	"411",	"421",	"422",	"429",	"431",	"431",	"432",	"432",	"432",	"433",	"439",	"451",	"451",	"452",	"453",	"454",	"454",	"461",	"462",	"463",	"463",	"464",	"464",	"464",	"464",	"464",	"464",	"465",	"465",	"465",	"465",	"466",	"466",	"466",	"466",	"466",	"466",	"469",	"471",	"471",	"472",	"472",	"472",	"472",	"472",	"473",	"473",	"474",	"474",	"475",	"475",	"475",	"475",	"475",	"475",	"476",	"476",	"476",	"477",	"477",	"477",	"477",	"477",	"478",	"478",	"478",	"479",	"479",	"479",	"491",	"491",	"492",	"492",	"492",	"493",	"501",	"501",	"502",	"502",	"511",	"511",	"512",	"512",	"521",	"522",	"522",	"522",	"522",	"522",	"531",	"532",	"551",	"551",	"551",	"551",	"551",	"552",	"553",	"559",	"561",	"561",	"561",	"561",	"562",	"562",	"563",	"581",	"581",	"581",	"581",	"582",	"591",	"591",	"591",	"591",	"592",	"601",	"602",	"611",	"612",	"613",	"619",	"620",	"620",	"620",	"631",	"631",	"639",	"639",	"641",	"641",	"642",	"642",	"642",	"642",	"643",	"643",	"649",	"649",	"649",	"649",	"649",	"649",	"651",	"651",	"651",	"651",	"652",	"652",	"653",	"653",	"661",	"661",	"661",	"661",	"661",	"661",	"662",	"662",	"663",	"681",	"682",	"691",	"692",	"701",	"702",	"711",	"712",	"721",	"722",	"731",	"732",	"741",	"742",	"749",	"750",	"771",	"772",	"772",	"772",	"773",	"774",	"781",	"782",	"783",	"791",	"791",	"799",	"801",	"802",	"803",	"811",	"812",	"812",	"813",	"821",	"821",	"822",	"823",	"829",	"829",	"829",	"841",	"841",	"841",	"841",	"841",	"842",	"842",	"842",	"842",	"843",	"851",	"851",	"851",	"852",	"852",	"852",	"853",	"854",	"854",	"854",	"854",	"855",	"855",	"855",	"855",	"856",	"861",	"862",	"862",	"869",	"869",	"869",	"871",	"872",	"873",	"879",	"881",	"889",	"900",	"900",	"900",	"900",	"900",	"900",	"900",	"900",	"910",	"910",	"910",	"920",	"931",	"931",	"931",	"932",	"932",	"941",	"941",	"942",	"949",	"949",	"949",	"951",	"951",	"952",	"952",	"952",	"952",	"952",	"960",	"960",	"960",	"960",	"970",	"981",	"982",	"990",	"011",	"011",	"011",	"011",	"011",	"011",	"012",	"012",	"012",	"012",	"012",	"012",	"012",	"012",	"012",	"013",	"014",	"014",	"014",	"014",	"014",	"014",	"015",	"016",	"016",	"016",	"016",	"017",	"021",	"022",	"023",	"024",	"031",	"031",	"032",	"032",	"051",	"052",	"061",	"062",	"071",	"072",	"072",	"072",	"072",	"081",	"081",	"82",	"089",	"089",	"089",	"091",	"099",	"711",	"711",	"081"],

        'DES_GRU': ["Comercio al por menor de alimentos (víveres en general), bebidas y tabaco, en establecimientos especializados",	"Actividades de servicios de apoyo a las empresas n.c.p.",	"Otras actividades de atención relacionadas con la salud humana",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Propagación de plantas (actividades de los viveros, excepto viveros forestales) ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Explotación mixta (agrícola y pecuaria) ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Silvicultura y otras actividades forestales",	"Extracción de madera ",	"Servicios de apoyo a la silvicultura ",	"Pesca ",	"Pesca ",	"Acuicultura ",	"Acuicultura ",	"Extracción de hulla (carbón de piedra)",	"Extracción de minerales de hierro",	"Extracción de minerales metalíferos no ferrosos",	"Extracción de minerales metalíferos no ferrosos",	"Extracción de piedra, arena, arcillas, cal, yeso, caolín, bentonitas y similares",	"Extracción de piedra, arena, arcillas, cal, yeso, caolín, bentonitas y similares",	"Extracción de esmeraldas, piedras preciosas y semipreciosas",	"Extracción de otros minerales no metálicos n.c.p.",	"Actividades de apoyo para la extracción de petróleo y de gas natural",	"Actividades de apoyo para otras actividades de explotación de minas y canteras",	"Procesamiento y conservación de carne, pescado, crustáceos y moluscos ",	"Procesamiento y conservación de carne, pescado, crustáceos y moluscos ",	"Procesamiento y conservación de frutas, legumbres, hortalizas y tubérculos",	"Elaboración de aceites y grasas de origen vegetal y animal",	"Elaboración de productos lácteos",	"Elaboración de productos de molinería, almidones y productos derivados del almidón",	"Elaboración de productos de molinería, almidones y productos derivados del almidón",	"Elaboración de productos de café",	"Elaboración de productos de café",	"Elaboración de productos de café",	"Elaboración de azúcar y panela",	"Elaboración de azúcar y panela",	"Elaboración de otros productos alimenticios",	"Elaboración de otros productos alimenticios",	"Elaboración de otros productos alimenticios",	"Elaboración de otros productos alimenticios",	"Elaboración de otros productos alimenticios",	"Elaboración de alimentos preparados para animales",	"Elaboración de bebidas",	"Elaboración de bebidas",	"Elaboración de bebidas",	"Elaboración de bebidas",	"Elaboración de productos de tabaco",	"Preparación, hilatura, tejeduría y acabado de productos textiles",	"Preparación, hilatura, tejeduría y acabado de productos textiles",	"Preparación, hilatura, tejeduría y acabado de productos textiles",	"Fabricación de otros productos textiles",	"Fabricación de otros productos textiles",	"Fabricación de otros productos textiles",	"Fabricación de otros productos textiles",	"Fabricación de otros productos textiles",	"Confección de prendas de vestir, excepto prendas de piel",	"Fabricación de artículos de piel",	"Fabricación de artículos de punto y ganchillo",	"Curtido y recurtido de cueros; fabricación de artículos de viaje, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y guarnicionería, adobo y teñido de pieles",	"Curtido y recurtido de cueros; fabricación de artículos de viaje, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y guarnicionería, adobo y teñido de pieles",	"Curtido y recurtido de cueros; fabricación de artículos de viaje, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y guarnicionería, adobo y teñido de pieles",	"Fabricación de calzado",	"Fabricación de calzado",	"Fabricación de calzado",	"Aserrado, acepillado e impregnación de la madera",	"Fabricación de hojas de madera para enchapado; fabricación de tableros contrachapados, tableros laminados, tableros de partículas y otros tableros y paneles",	"Fabricación de partes y piezas de madera, de carpintería y ebanistería para la construcción",	"Fabricación de recipientes de madera",	"Fabricación de otros productos de madera; fabricación de artículos de corcho, cestería y espartería",	"Fabricación de papel, cartón y productos de papel y cartón",	"Fabricación de papel, cartón y productos de papel y cartón",	"Fabricación de papel, cartón y productos de papel y cartón",	"Actividades de impresión y actividades de servicios relacionados con la impresión",	"Actividades de impresión y actividades de servicios relacionados con la impresión",	"Producción de copias a partir de grabaciones originales ",	"Fabricación de productos de hornos de coque",	"Fabricación de productos de la refinación del petróleo",	"Fabricación de productos de la refinación del petróleo",	"Fabricación de sustancias químicas básicas, abonos y compuestos inorgánicos nitrogenados, plásticos y caucho sintético en formas primarias",	"Fabricación de sustancias químicas básicas, abonos y compuestos inorgánicos nitrogenados, plásticos y caucho sintético en formas primarias",	"Fabricación de sustancias químicas básicas, abonos y compuestos inorgánicos nitrogenados, plásticos y caucho sintético en formas primarias",	"Fabricación de sustancias químicas básicas, abonos y compuestos inorgánicos nitrogenados, plásticos y caucho sintético en formas primarias",	"Fabricación de otros productos químicos",	"Fabricación de otros productos químicos",	"Fabricación de otros productos químicos",	"Fabricación de otros productos químicos",	"Fabricación de fibras sintéticas y artificiales",	"Fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos de uso farmacéutico",	"Fabricación de productos de caucho",	"Fabricación de productos de caucho",	"Fabricación de productos de caucho",	"Fabricación de productos de plástico",	"Fabricación de productos de plástico",	"Fabricación de vidrio y productos de vidrio",	"Fabricación de productos minerales no metálicos n.c.p.",	"Fabricación de productos minerales no metálicos n.c.p.",	"Fabricación de productos minerales no metálicos n.c.p.",	"Fabricación de productos minerales no metálicos n.c.p.",	"Fabricación de productos minerales no metálicos n.c.p.",	"Fabricación de productos minerales no metálicos n.c.p.",	"Fabricación de productos minerales no metálicos n.c.p.",	"Industrias básicas de hierro y de acero",	"Industrias básicas de metales preciosos y de metales no ferrosos",	"Industrias básicas de metales preciosos y de metales no ferrosos",	"Fundición de metales",	"Fundición de metales",	"Fabricación de productos metálicos para uso estructural, tanques, depósitos y generadores de vapor",	"Fabricación de productos metálicos para uso estructural, tanques, depósitos y generadores de vapor",	"Fabricación de productos metálicos para uso estructural, tanques, depósitos y generadores de vapor",	"Fabricación de armas y municiones",	"Fabricación de otros productos elaborados de metal y actividades de servicios relacionadas con el trabajo de metales",	"Fabricación de otros productos elaborados de metal y actividades de servicios relacionadas con el trabajo de metales",	"Fabricación de otros productos elaborados de metal y actividades de servicios relacionadas con el trabajo de metales",	"Fabricación de otros productos elaborados de metal y actividades de servicios relacionadas con el trabajo de metales",	"Fabricación de componentes y tableros electrónicos",	"Fabricación de computadoras y de equipo periférico",	"Fabricación de equipos de comunicación",	"Fabricación de aparatos electrónicos de consumo",	"Fabricación de equipo de medición, prueba, navegación y control; fabricación de relojes",	"Fabricación de equipo de medición, prueba, navegación y control; fabricación de relojes",	"Fabricación de equipo de irradiación y equipo electrónico de uso médico y terapéutico",	"Fabricación de instrumentos ópticos y equipo fotográfico",	"Fabricación de medios magnéticos y ópticos para almacenamiento de datos",	"Fabricación de motores, generadores y transformadores eléctricos y de aparatos de distribución y control de la energía eléctrica",	"Fabricación de motores, generadores y transformadores eléctricos y de aparatos de distribución y control de la energía eléctrica",	"Fabricación de pilas, baterías y acumuladores eléctricos",	"Fabricación de hilos y cables aislados y sus dispositivos",	"Fabricación de hilos y cables aislados y sus dispositivos",	"Fabricación de equipos eléctricos de iluminación",	"Fabricación de aparatos de uso doméstico",	"Fabricación de otros tipos de equipo eléctrico n.c.p.",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso general",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de maquinaria y equipo de uso especial",	"Fabricación de vehículos automotores y sus motores",	"Fabricación de carrocerías para vehículos automotores; fabricación de remolques y semirremolques ",	"Fabricación de partes, piezas (autopartes) y accesorios (lujos) para vehículos automotores",	"Construcción de barcos y otras embarcaciones",	"Construcción de barcos y otras embarcaciones",	"Fabricación de locomotoras y de material rodante para ferrocarriles",	"Fabricación de aeronaves, naves espaciales y de maquinaria conexa",	"Fabricación de vehículos militares de combate",	"Fabricación de otros tipos de equipo de transporte n.c.p.",	"Fabricación de otros tipos de equipo de transporte n.c.p.",	"Fabricación de otros tipos de equipo de transporte n.c.p.",	"Fabricación de muebles ",	"Fabricación de colchones y somieres",	"Fabricación de joyas, bisutería y artículos conexos",	"Fabricación de instrumentos musicales",	"Fabricación de artículos y equipo para la práctica del deporte",	"Fabricación de juegos, juguetes y rompecabezas",	"Fabricación de instrumentos, aparatos y materiales médicos y odontológicos (incluido mobiliario)",	"Otras industrias manufactureras n.c.p.",	"Mantenimiento y reparación especializado de productos elaborados en metal y de maquinaria y equipo",	"Mantenimiento y reparación especializado de productos elaborados en metal y de maquinaria y equipo",	"Mantenimiento y reparación especializado de productos elaborados en metal y de maquinaria y equipo",	"Mantenimiento y reparación especializado de productos elaborados en metal y de maquinaria y equipo",	"Mantenimiento y reparación especializado de productos elaborados en metal y de maquinaria y equipo",	"Mantenimiento y reparación especializado de productos elaborados en metal y de maquinaria y equipo",	"Instalación especializada de maquinaria y equipo industrial ",	"Generación, transmisión, distribución y comercialización de energía eléctrica",	"Generación, transmisión, distribución y comercialización de energía eléctrica",	"Generación, transmisión, distribución y comercialización de energía eléctrica",	"Generación, transmisión, distribución y comercialización de energía eléctrica",	"Producción de gas; distribución de combustibles gaseosos por tuberías",	"Suministro de vapor y aire acondicionado",	"Captación, tratamiento y distribución de agua",	"Evacuación y tratamiento de aguas residuales",	"Recolección de desechos",	"Recolección de desechos",	"Tratamiento y disposición de desechos",	"Tratamiento y disposición de desechos",	"Recuperación de materiales",	"Actividades de saneamiento ambiental y otros servicios de gestión de desechos",	"Construcción de edificios",	"Construcción de edificios",	"Construcción de carreteras y vías de ferrocarril",	"Construcción de proyectos de servicio público",	"Construcción de otras obras de ingeniería civil",	"Demolición y preparación del terreno",	"Demolición y preparación del terreno",	"Instalaciones eléctricas, de fontanería y otras instalaciones especializadas",	"Instalaciones eléctricas, de fontanería y otras instalaciones especializadas",	"Instalaciones eléctricas, de fontanería y otras instalaciones especializadas",	"Terminación y acabado de edificios y obras de ingeniería civil",	"Otras actividades especializadas para la construcción de edificios y obras de ingeniería civil",	"Comercio de vehículos automotores",	"Comercio de vehículos automotores",	"Mantenimiento y reparación de vehículos automotores",	"Comercio de partes, piezas (autopartes) y accesorios (lujos) para vehículos automotores",	"Comercio, mantenimiento y reparación de motocicletas y de sus partes, piezas y accesorios",	"Comercio, mantenimiento y reparación de motocicletas y de sus partes, piezas y accesorios",	"Comercio al por mayor a cambio de una retribución o por contrata",	"Comercio al por mayor de materias primas agropecuarias; animales vivos",	"Comercio al por mayor de alimentos, bebidas y tabaco",	"Comercio al por mayor de alimentos, bebidas y tabaco",	"Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir)",	"Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir)",	"Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir)",	"Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir)",	"Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir)",	"Comercio al por mayor de artículos y enseres domésticos (incluidas prendas de vestir)",	"Comercio al por mayor de maquinaria y equipo ",	"Comercio al por mayor de maquinaria y equipo ",	"Comercio al por mayor de maquinaria y equipo ",	"Comercio al por mayor de maquinaria y equipo ",	"Comercio al por mayor especializado de otros productos",	"Comercio al por mayor especializado de otros productos",	"Comercio al por mayor especializado de otros productos",	"Comercio al por mayor especializado de otros productos",	"Comercio al por mayor especializado de otros productos",	"Comercio al por mayor especializado de otros productos",	"Comercio al por mayor no especializado",	"Comercio al por menor en establecimientos no especializados",	"Comercio al por menor en establecimientos no especializados",	"Comercio al por menor de alimentos (víveres en general), bebidas y tabaco, en establecimientos especializados",	"Comercio al por menor de alimentos (víveres en general), bebidas y tabaco, en establecimientos especializados",	"Comercio al por menor de alimentos (víveres en general), bebidas y tabaco, en establecimientos especializados",	"Comercio al por menor de alimentos (víveres en general), bebidas y tabaco, en establecimientos especializados",	"Comercio al por menor de alimentos (víveres en general), bebidas y tabaco, en establecimientos especializados",	"Comercio al por menor de combustible, lubricantes, aditivos y productos de limpieza para automotores, en establecimientos especializados",	"Comercio al por menor de combustible, lubricantes, aditivos y productos de limpieza para automotores, en establecimientos especializados",	"Comercio al por menor de equipos de informática y de comunicaciones, en establecimientos especializados",	"Comercio al por menor de equipos de informática y de comunicaciones, en establecimientos especializados",	"Comercio al por menor de otros enseres domésticos en establecimientos especializados",	"Comercio al por menor de otros enseres domésticos en establecimientos especializados",	"Comercio al por menor de otros enseres domésticos en establecimientos especializados",	"Comercio al por menor de otros enseres domésticos en establecimientos especializados",	"Comercio al por menor de otros enseres domésticos en establecimientos especializados",	"Comercio al por menor de otros enseres domésticos en establecimientos especializados",	"Comercio al por menor de artículos culturales y de entretenimiento, en establecimientos especializados",	"Comercio al por menor de artículos culturales y de entretenimiento, en establecimientos especializados",	"Comercio al por menor de artículos culturales y de entretenimiento, en establecimientos especializados",	"Comercio al por menor de otros productos en establecimientos especializados",	"Comercio al por menor de otros productos en establecimientos especializados",	"Comercio al por menor de otros productos en establecimientos especializados",	"Comercio al por menor de otros productos en establecimientos especializados",	"Comercio al por menor de otros productos en establecimientos especializados",	"Comercio al por menor en puestos de venta móviles",	"Comercio al por menor en puestos de venta móviles",	"Comercio al por menor en puestos de venta móviles",	"Comercio al por menor no realizado en establecimientos, puestos de venta o mercados",	"Comercio al por menor no realizado en establecimientos, puestos de venta o mercados",	"Comercio al por menor no realizado en establecimientos, puestos de venta o mercados",	"Transporte férreo",	"Transporte férreo",	"Transporte terrestre público automotor",	"Transporte terrestre público automotor",	"Transporte terrestre público automotor",	"Transporte por tuberías",	"Transporte marítimo y de cabotaje",	"Transporte marítimo y de cabotaje",	"Transporte fluvial",	"Transporte fluvial",	"Transporte aéreo de pasajeros ",	"Transporte aéreo de pasajeros ",	"Transporte aéreo de carga ",	"Transporte aéreo de carga ",	"Almacenamiento y depósito",	"Actividades de las estaciones, vías y servicios complementarios para el transporte",	"Actividades de las estaciones, vías y servicios complementarios para el transporte",	"Actividades de las estaciones, vías y servicios complementarios para el transporte",	"Actividades de las estaciones, vías y servicios complementarios para el transporte",	"Actividades de las estaciones, vías y servicios complementarios para el transporte",	"Actividades postales nacionales",	"Actividades de mensajería",	"Actividades de alojamiento de estancias cortas",	"Actividades de alojamiento de estancias cortas",	"Actividades de alojamiento de estancias cortas",	"Actividades de alojamiento de estancias cortas",	"Actividades de alojamiento de estancias cortas",	"Actividades de zonas de camping y parques para vehículos recreacionales",	"Servicio por horas ",	"Otros tipos de alojamiento n.c.p.",	"Actividades de restaurantes, cafeterías y servicio móvil de comidas",	"Actividades de restaurantes, cafeterías y servicio móvil de comidas",	"Actividades de restaurantes, cafeterías y servicio móvil de comidas",	"Actividades de restaurantes, cafeterías y servicio móvil de comidas",	"Actividades de catering para eventos y otros servicios de comidas",	"Actividades de catering para eventos y otros servicios de comidas",	"Expendio de bebidas alcohólicas para el consumo dentro del establecimiento",	"Edición de libros, publicaciones periódicas y otras actividades de edición",	"Edición de libros, publicaciones periódicas y otras actividades de edición",	"Edición de libros, publicaciones periódicas y otras actividades de edición",	"Edición de libros, publicaciones periódicas y otras actividades de edición",	"Edición de programas de informática (software)",	"Actividades de producción de películas cinematográficas, video y producción de programas, anuncios y comerciales de televisión",	"Actividades de producción de películas cinematográficas, video y producción de programas, anuncios y comerciales de televisión",	"Actividades de producción de películas cinematográficas, video y producción de programas, anuncios y comerciales de televisión",	"Actividades de producción de películas cinematográficas, video y producción de programas, anuncios y comerciales de televisión",	"Actividades de grabación de sonido y edición de música",	"Actividades de programación y transmisión en el servicio de radiodifusión sonora",	"Actividades de programación y transmisión de televisión",	"Actividades de telecomunicaciones alámbricas",	"Actividades de telecomunicaciones inalámbricas",	"Actividades de telecomunicación satelital",	"Otras actividades de telecomunicaciones",	"Desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas), consultoría informática y actividades relacionadas",	"Desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas), consultoría informática y actividades relacionadas",	"Desarrollo de sistemas informáticos (planificación, análisis, diseño, programación, pruebas), consultoría informática y actividades relacionadas",	"Procesamiento de datos, alojamiento (hosting) y actividades relacionadas; portales web",	"Procesamiento de datos, alojamiento (hosting) y actividades relacionadas; portales web",	"Otras actividades de servicio de información",	"Otras actividades de servicio de información",	"Intermediación monetaria",	"Intermediación monetaria",	"Otros tipos de intermediación monetaria",	"Otros tipos de intermediación monetaria",	"Otros tipos de intermediación monetaria",	"Otros tipos de intermediación monetaria",	"Fideicomisos, fondos (incluye fondos de cesantías) y entidades financieras similares",	"Fideicomisos, fondos (incluye fondos de cesantías) y entidades financieras similares",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones",	"Otras actividades de servicio financiero, excepto las de seguros y pensiones",	"Seguros y capitalización",	"Seguros y capitalización",	"Seguros y capitalización",	"Seguros y capitalización",	"Servicios de seguros sociales de salud y riesgos profesionales",	"Servicios de seguros sociales de salud y riesgos profesionales",	"Servicios de seguros sociales de pensiones",	"Servicios de seguros sociales de pensiones",	"Actividades auxiliares de las actividades de servicios financieros, excepto las de seguros y pensiones",	"Actividades auxiliares de las actividades de servicios financieros, excepto las de seguros y pensiones",	"Actividades auxiliares de las actividades de servicios financieros, excepto las de seguros y pensiones",	"Actividades auxiliares de las actividades de servicios financieros, excepto las de seguros y pensiones",	"Actividades auxiliares de las actividades de servicios financieros, excepto las de seguros y pensiones",	"Actividades auxiliares de las actividades de servicios financieros, excepto las de seguros y pensiones",	"Actividades de servicios auxiliares de los servicios de seguros y pensiones",	"Actividades de servicios auxiliares de los servicios de seguros y pensiones",	"Actividades de administración de fondos",	"Actividades inmobiliarias realizadas con bienes propios o arrendados",	"Actividades inmobiliarias realizadas a cambio de una retribución o por contrata ",	"Actividades jurídicas",	"Actividades de contabilidad, teneduría de libros, auditoría financiera y asesoría tributaria",	"Actividades de administración empresarial",	"Actividades de consultaría de gestión",	"Actividades de arquitectura e ingeniería y otras actividades conexas de consultoría técnica",	"Ensayos y análisis técnicos",	"Investigaciones y desarrollo experimental en el campo de las ciencias naturales y la ingeniería ",	"Investigaciones y desarrollo experimental en el campo de las ciencias sociales y las humanidades",	"Publicidad",	"Estudios de mercado y realización de encuestas de opinión pública",	"Actividades especializadas de diseño ",	"Actividades de fotografía",	"Otras actividades profesionales, científicas y técnicas n.c.p.",	"Actividades veterinarias",	"Alquiler y arrendamiento de vehículos automotores",	"Alquiler y arrendamiento de efectos personales y enseres domésticos",	"Alquiler y arrendamiento de efectos personales y enseres domésticos",	"Alquiler y arrendamiento de efectos personales y enseres domésticos",	"Alquiler y arrendamiento de otros tipos de maquinaria, equipo y bienes tangibles n.c.p.",	"Arrendamiento de propiedad intelectual y productos similares, excepto obras protegidas por derechos de autor",	"Actividades de agencias de empleo",	"Actividades de agencias de empleo temporal",	"Otras actividades de suministro de recurso humano",	"Actividades de las agencias de viajes y operadores turísticos",	"Actividades de las agencias de viajes y operadores turísticos",	"Otros servicios de reserva y actividades relacionadas",	"Actividades de seguridad privada",	"Actividades de servicios de sistemas de seguridad",	"Actividades de detectives e investigadores privados",	"Actividades combinadas de apoyo a instalaciones",	"Actividades de limpieza",	"Actividades de limpieza",	"Actividades de paisajismo y servicios de mantenimiento conexos",	"Actividades administrativas y de apoyo de oficina",	"Actividades administrativas y de apoyo de oficina",	"Actividades de centros de llamadas (Call center)",	"Organización de convenciones y eventos comerciales",	"Actividades de servicios de apoyo a las empresas n.c.p.",	"Actividades de servicios de apoyo a las empresas n.c.p.",	"Actividades de servicios de apoyo a las empresas n.c.p.",	"Administración del Estado y aplicación de la política económica y social de la comunidad",	"Administración del Estado y aplicación de la política económica y social de la comunidad",	"Administración del Estado y aplicación de la política económica y social de la comunidad",	"Administración del Estado y aplicación de la política económica y social de la comunidad",	"Administración del Estado y aplicación de la política económica y social de la comunidad",	"Prestación de servicios a la comunidad en general",	"Prestación de servicios a la comunidad en general",	"Prestación de servicios a la comunidad en general",	"Prestación de servicios a la comunidad en general",	"Actividades de planes de seguridad social de afiliación obligatoria",	"Educación de la primera infancia, preescolar y básica primaria",	"Educación de la primera infancia, preescolar y básica primaria",	"Educación de la primera infancia, preescolar y básica primaria",	"Educación secundaria y de formación laboral",	"Educación secundaria y de formación laboral",	"Educación secundaria y de formación laboral",	"Establecimientos que combinan diferentes niveles de educación ",	"Educación superior",	"Educación superior",	"Educación superior",	"Educación superior",	"Otros tipos de educación",	"Otros tipos de educación",	"Otros tipos de educación",	"Otros tipos de educación",	"Actividades de apoyo a la educación",	"Actividades de hospitales y clínicas, con internación",	"Actividades de práctica médica y odontológica, sin internación ",	"Actividades de práctica médica y odontológica, sin internación ",	"Otras actividades de atención relacionadas con la salud humana",	"Otras actividades de atención relacionadas con la salud humana",	"Otras actividades de atención relacionadas con la salud humana",	"Actividades de atención residencial medicalizada de tipo general",	"Actividades de atención residencial, para el cuidado de pacientes con retardo mental, enfermedad mental y consumo de sustancias psicoactivas",	"Actividades de atención en instituciones para el cuidado de personas mayores y/o discapacitadas",	"Otras actividades de atención en instituciones con alojamiento",	"Actividades de asistencia social sin alojamiento para personas mayores y discapacitadas",	"Otras actividades de asistencia social sin alojamiento",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades creativas, artísticas y de entretenimiento ",	"Actividades de bibliotecas, archivos, museos y otras actividades culturales",	"Actividades de bibliotecas, archivos, museos y otras actividades culturales",	"Actividades de bibliotecas, archivos, museos y otras actividades culturales",	"Actividades de juegos de azar y apuestas",	"Actividades deportivas",	"Actividades deportivas",	"Actividades deportivas",	"Otras actividades recreativas y de esparcimiento",	"Otras actividades recreativas y de esparcimiento",	"Actividades de asociaciones empresariales y de empleadores, ",	"Actividades de asociaciones empresariales y de empleadores, ",	"Actividades de sindicatos de empleados",	"Actividades de otras asociaciones",	"Actividades de otras asociaciones",	"Actividades de otras asociaciones",	"Mantenimiento y reparación de computadores y equipo de comunicaciones",	"Mantenimiento y reparación de computadores y equipo de comunicaciones",	"Mantenimiento y reparación de efectos personales y enseres domésticos",	"Mantenimiento y reparación de efectos personales y enseres domésticos",	"Mantenimiento y reparación de efectos personales y enseres domésticos",	"Mantenimiento y reparación de efectos personales y enseres domésticos",	"Mantenimiento y reparación de efectos personales y enseres domésticos",	"Otras actividades de servicios personales",	"Otras actividades de servicios personales",	"Otras actividades de servicios personales",	"Otras actividades de servicios personales",	"Actividades de los hogares individuales como empleadores de personal doméstico",	"Actividades no diferenciadas de los hogares individuales como productores de bienes para uso propio",	"Actividades no diferenciadas de los hogares individuales como productores de servicios para uso propio",	"Actividades de organizaciones y entidades extraterritoriales",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas transitorios ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Cultivos agrícolas permanentes ",	"Propagación de plantas (actividades de los viveros, excepto viveros forestales) ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Ganadería ",	"Explotación mixta (agrícola y pecuaria) ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Actividades de apoyo a la agricultura y la ganadería, y actividades posteriores a la cosecha ",	"Caza ordinaria y mediante trampas y actividades de servicios conexas ",	"Silvicultura y otras actividades forestales",	"Extracción de madera ",	"Recolección de productos forestales diferentes a la madera",	"Servicios de apoyo a la silvicultura ",	"Pesca ",	"Pesca ",	"Acuicultura ",	"Acuicultura ",	"Extracción de hulla (carbón de piedra)",	"Extracción de carbón lignito",	"Extracción de petróleo crudo",	"Extracción de gas natural",	"Extracción de minerales de hierro",	"Extracción de minerales metalíferos no ferrosos",	"Extracción de minerales metalíferos no ferrosos",	"Extracción de minerales metalíferos no ferrosos",	"Extracción de minerales metalíferos no ferrosos",	"Extracción de piedra, arena, arcillas, cal, yeso, caolín, bentonitas y similares",	"Extracción de piedra, arena, arcillas, cal, yeso, caolín, bentonitas y similares",	"Extracción de esmeraldas, piedras preciosas y semipreciosas",	"Extracción de otros minerales no metálicos n.c.p.",	"Extracción de otros minerales no metálicos n.c.p.",	"Extracción de otros minerales no metálicos n.c.p.",	"Actividades de apoyo para la extracción de petróleo y de gas natural",	"Actividades de apoyo para otras actividades de explotación de minas y canteras",	"Actividades de arquitectura e ingeniería y otras actividades conexas de consultoría técnica",	"Actividades de arquitectura e ingeniería y otras actividades conexas de consultoría técnica",	"Extracción de minerales para la fabricación de abonos y productos químicos"],

        }

In [ ]:
# Se crea el dataframe con el diccionario de datos referente a los CIIU para el posterior cruce de información
df_CIIU = pd.DataFrame(dict_CIIU, columns = ["CODIGO_CIIU",	"CIIU_COR",	"DESC_CIIU_COR", "SECCION_COR",	"DESC_SECCION_COR",	"DIVISION",	"DES_DIVI",	"GRUPO", "DES_GRU"])
df_CIIU.to_excel(r'dataframeciius.xlsx', index = False)

In [ ]:
# Para la correcta unión entre las bases se procede establecer el tipo llave que se utilizará "str"
df3.CODIGO_CIIU = df3.CODIGO_CIIU.astype(str)
df_CIIU.CODIGO_CIIU = df_CIIU.CODIGO_CIIU.astype(str)

In [ ]:
# Se unen las dos bases de datos para obtener la información con código CIIU corregidos
df3 = pd.merge(df3, df_CIIU, how='left', on=['CODIGO_CIIU'], indicator="indicator_column")

<p>----</p>

In [ ]:
# Siguiendo las indicaciones de Sistemas y Registro se deben eliminar las filas de las empresas que registren en el ESTADO_MAT_REN la letra R ya que estos movimientos fueron anulados por alguna acción legal. En la mayoría de los casos se anulan las renovaciones por motivo de cancelación hechas antes del 31 de marzo de cada año ya que la ley permite que las empresas cancelen su registro sin pagar la renovación solo si se realiza el proceso antes del 31 de marzo de cda año
df4 = df3.query('ESTADO_MAT_REN!="R"')

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
df4.to_excel(r'MATREM_CCC_ENE-AGO2022_V3(SEP_20_EEE3).xlsx', index = False)
#df4.to_csv(r'RM_FEB2021_V2(MAR2021_EEE3).csv', index = False, encoding='utf-8-sig')